# Database Creation

In [1]:
import calendar
import glob
import json
import os
import re
import sys
from collections import defaultdict, namedtuple
from datetime import datetime
from functools import partial
from io import StringIO
from time import sleep, time
from typing import Callable

import numpy as np
import pandas as pd
import pandasql as ps
import requests
from bs4 import BeautifulSoup, Comment
from geopy.geocoders import Nominatim
from geopy.location import Location
from pybaseball import playerid_reverse_lookup, team_ids
from selenium import webdriver
from tqdm.auto import trange, tqdm
from unidecode import unidecode

## Franchises and Teams

In [2]:
SCRAPE = False

STATE_DICT = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia', 'AS': 'American Samoa', 'GU': 'Guam', 'MP': 'Northern Mariana Islands', 'PR': 'Puerto Rico', 'UM': 'United States Minor Outlying Islands', 'VI': 'U.S. Virgin Islands', 'NSW': 'New South Wales', 'QLD': 'Queensland', 'AB': 'Alberta', 'BC': 'British Columbia', 'MB': 'Manitoba', 'NB': 'New Brunswick', 'ON': 'Ontario', 'PE': 'Prince Edward Island', 'PQ': 'Quebec', 'QC': 'Quebec', 'SK': 'Saskatchewan', 'SL': 'San Luis Potosí', 'SO': 'Sonora', 'VE': 'Veracruz'}

COUNTRY_DICT = {'USA': 'United States', 'VEN': 'Venezuela', 'DOM': 'Dominican Republic', 'MEX': 'Mexico', 'NIR': 'Northern Ireland', 'USSR': 'Soviet Union', 'NCA': 'Nicaragua', 'CUB': 'Cuba', 'PUR': 'Puerto Rico'}

FG_TO_LAHMAN_FRANCH_MAPPING = {16: 'ATL', 17: 'CHC', 1002: 'PNA', 1005: 'BLC', 1010: 'BRA', 1019: 'CEN', 1020: 'CFC', 1032: 'ECK', 1033: 'HAR', 1042: 'KEK', 1045: 'MAN', 1046: 'MAR', 1050: 'NAT', 1052: 'NHV', 1053: 'NYU', 1057: 'OLY', 1065: 'PWS', 1066: 'RES', 1069: 'ROK', 1070: 'SBS', 1073: 'SLR', 1080: 'TRO', 1083: 'WBL', 1084: 'WES', 1088: 'WNT'}

FG_TO_BR_FRANCH_MAPPING = FG_TO_LAHMAN_FRANCH_MAPPING | {1033: 'HNA', 1053: 'NNA', 1070: 'SNA'}

FG_TO_BR_NA_TEAM_MAPPING = pd.DataFrame.from_dict({16: 'BOS', 17: 'CHI', 1002: 'ATH', 1005: 'BAL', 1010: 'BRA', 1019: 'CEN', 1020: 'CLE', 1032: 'ECK', 1033: 'HAR', 1042: 'KEK', 1045: 'MAN', 1046: 'MAR', 1050: 'NAT', 1052: 'NHV', 1053: 'NYU', 1057: 'OLY', 1065: 'PHI', 1066: 'RES', 1069: 'ROK', 1070: 'STL', 1073: 'SLR', 1080: 'TRO', 1083: 'WAS', 1084: 'WES', 1088: 'WAS'}, orient='index', columns=['teamAbrBR'])

LAHMAN_CORRECTIONS = {'ATH': 'PNA', 'BNA': 'ATL', 'CNA': 'CHC', 'HNA': 'HAR', 'NNA': 'NYU', 'SNA': 'SBS'}

BR_FRANCH_CORRECTIONS = {'NYU': 'NNA', 'HAR': 'HNA', 'STL': 'SNA'}

BR_TO_LAHMAN = [1003, 1004, 1007, 1011, 1013, 1017, 1018, 1021, 1022, 1024, 1025, 1028, 1035, 1036, 1041, 1043, 1047, 1049, 1062, 1071, 1077, 1078, 1081, 1082, 1086, 1087, 1090]

BREF_HANDEDNESS = defaultdict(lambda: 'R')
BREF_HANDEDNESS['*'] = 'L'
BREF_HANDEDNESS['#'] = 'B'
BREF_HANDEDNESS['?'] = np.nan

INT_COLS = {'Year', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'W', 'L', 'GS', 'GF', 'CG', 'SHO', 'SV', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'BK', 'WP', 'BF', 'ERA+', 'Ch', 'PO', 'A', 'E', 'DP', 'Rtot', 'Rdrs', 'Rtot/yr', 'Rdrs/yr', 'PK', 'Rbat', 'Rbaser', 'Rdp', 'Rfield', 'Rpos', 'RAA', 'Rrep', 'RAR', 'oRAR', 'Rbat+'}
FLOAT_COLS = {'BA', 'OBP', 'SLG', 'OPS', 'W-L%', 'ERA', 'IP', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Inn', 'Fld%', 'RF/9', 'RF/G', 'WAA', 'WAR', 'waaWL%', '162WL%', 'oWAR', 'dWAR', 'RA9', 'RA9opp', 'RA9def', 'RA9role', 'PPFp', 'RA9avg', 'gmLI', 'WAAadj', 'rOBA', 'BAbip', 'ISO', 'GB/FB', 'WPA', 'RE24'}


def match_location(loc_dict: dict[str, str], location: str | None = None, geocode: Callable[[str], Location] | None = None, raw_address: dict[str, str] | None = None) -> dict[str, str]:
    if not raw_address:
        if location and geocode:
            raw_address = geocode(location).raw['address']
        else:
            return

    for city in {'city', 'town', 'village'} & raw_address.keys():
        loc_dict['city'] = raw_address[city]
    for state in {'state', 'province'} & raw_address.keys():
        loc_dict['state'] = raw_address[state]
    for country in {'country', 'place'} & raw_address.keys():
        loc_dict['country'] = raw_address[country]
        

def fangraphs_team_scrape(stats: str = 'bat') -> pd.DataFrame:
    url = f'https://www.fangraphs.com/leaders/major-league?pos=all&stats={stats}&lg=aa&lg=al&lg=fl&lg=na&lg=nl&lg=pl&lg=ua&qual=0&type=8&month=0&ind=0&team=0%2Cts&rost=&age=&filter=&players=0&startdate=&enddate=' + '&season1={season}&season={season}&pagenum=1&pageitems=2000000000'
    df = []

    for season in trange(1871, 2024):
        tdf = pd.DataFrame()
        while tdf.empty:
            try:
                tdf = pd.DataFrame(json.loads(BeautifulSoup(requests.get(url.format(season=season)).content, 'html.parser', from_encoding='utf_8').find('script', type='application/json').text)['props']['pageProps']['dehydratedState']['queries'][0]['state']['data']['data']).replace('- - -', np.nan).drop(['Name', 'PlayerName', 'PlayerNameRoute', 'Team', 'TeamNameAbb', 'SeasonMin', 'SeasonMax', 'Bats', 'Throws', 'xMLBAMID', 'playerid', 'Age', 'AgeR', 'G', 'GS', 'PA', 'IP', 'Pos', 'Inn'], axis=1, errors='ignore').rename({'TG': 'G', 'TPA': 'PA', 'TIP': 'IP', 'TInn': 'Inn'}, axis=1)
            except AttributeError:
                pass
            sleep(3)

        df.append(tdf)
    
    df = pd.concat(df, ignore_index=True)

    for col in {'IP', 'Inn'} & set(df.columns):
        df[col] = df[col].mul(3).round().div(3)

    return df

def scrape_table(io: str, match: str = '.+', index_col: str | None = None, extract_links: bool = True, commented: bool = False) -> pd.DataFrame:
    if commented:
        io = StringIO(str(BeautifulSoup([comment.strip() for comment in BeautifulSoup(requests.get(io).content, 'html.parser', from_encoding='utf_8').find_all(string=lambda text: isinstance(text, Comment)) if 'table' in comment and bool(re.search(match, comment))][0]).find('table')))
    return pd.read_html(io=io, match=match, flavor='bs4', index_col=index_col, extract_links='body' if extract_links else None)[0]


def bref_leaderboard_scrape(table: str, start: int = 1871, end: int = 2023, drop_tot: bool = True, commented: bool = False) -> pd.DataFrame:
    def scrape_page(table: str, season: int, league: str | None = None, commented: bool = False) -> pd.DataFrame:
        url = f'https://www.baseball-reference.com/leagues/{league if league else 'majors'}/{season}-{table.lower().replace(' ', '-')}.shtml'
        tdf = pd.DataFrame()
        while tdf.empty:
            try:
                if not commented:
                    try:
                        tdf = scrape_table(io=url, match=f'Player {table} Table', commented=commented)
                    except ValueError:
                        commented = True
                if commented:
                    tdf = scrape_table(io=url, match=f'Player {table} Table', commented=commented)

                tdf.insert(loc=tdf.columns.get_loc('Name') + 1, column='Season', value=[(season, None)] * tdf.shape[0])
            except AttributeError:
                pass
            sleep(3)
            
        return tdf
        
    df = []
    for season in trange(start, end + 1):
        if 1920 <= season <= 1948:
            df.extend([scrape_page(table=table, season=season, league='AL', commented=commented), scrape_page(table=table, season=season, league='NL', commented=commented)])
        else:
            df.append(scrape_page(table=table, season=season, commented=commented))

    df = pd.concat(df, ignore_index=True).drop(('Rk', None), errors='ignore').reset_index(drop=True)
    df.columns = [col.replace('\xa0', ' ') for col in df.columns]
    df = pd.concat([pd.DataFrame(df['Name'].tolist(), columns=['PlayerName', 'PlayerID']), df.drop(['Name', 'Age'], axis=1).apply(lambda col: col.str[0])], axis=1).rename({'Tm': 'Team', 'Lg': 'League', 'Pos Summary': 'Position'}, axis=1).query("Team != '" + ('TOT' if drop_tot else '') + "'").dropna(subset='PlayerID').apply(pd.to_numeric, errors='ignore')  # errors='ignore' is deprecated

    if not table.lower().endswith('fielding'):
        df.insert(loc=df.columns.get_loc('League') + 1, column='Bats' if table.lower().endswith('hitting') else 'Throws', value=df['PlayerName'].str[-1].map(BREF_HANDEDNESS))
    df['PlayerName'] = df['PlayerName'].str.replace('\xa0', ' ').str.strip('*#?')
    df['PlayerID'] = df['PlayerID'].str.extract(r'(?<=/[a-z]/)(.*)(?=\.shtml)')

    for col in {'IP', 'Inn'} & set(df.columns):
        df[col] = df[col].mul(3).round().div(3)

    return df

In [3]:
fields = {
    'name': {
        'roster_first': str,
        'roster_last': str,
        'given_first': str,
        'given_last': str,
        'middle': str
    }, 
    'bats': str, 
    'throws': str, 
    'height': {
        'feet': float, 
        'inches': int, 
        'meters': float
    }, 
    'weight': {
        'lbs': int,
        'kg': float
    }, 
    'born': {
        'year': int, 
        'month': int, 
        'day': int, 
        'country': str, 
        'state_province': str, 
        'city': str
    },
    'died': {
        'death_year': int,
        'death_month': int,
        'death_day': int,
        'death_country': str,
        'death_state_province': str,
        'death_city': str,
        'buried': {
                      'cemetery': str,
                      'city': str,
                      'state_province': str,
                      'country': str
                  }
    },
    'drafted': [
        {
            'team': str,
            'round': int,
            'pick': int,
            'year': int
        }
    ],
    'secondary_school': [
        {
            'name': str,
            'country': str,
            'state_province': str,
            'city': str
        }
    ],
    'tertiary_school': [
        {
            'name': str,
            'country': str,
            'state_province': str,
            'city': str
        }
    ],
    'debut': {
        'year': int, 
        'month': int, 
        'day': int
    },
    'finale': {
        'year': int,
        'month': int,
        'day': int
    },
    'year_rookie_eligibility_lost': int,
    'agency': [
        {
            'name': str,
            'current': bool
        }
    ]
}

In [6]:
tdf = pd.read_html(StringIO(str(BeautifulSoup([comment.strip() for comment in soup.find_all(string=lambda text: isinstance(text, Comment)) if 'table' in comment and bool(re.search('id="br-salaries"', comment))][0]).find('table'))))[0].dropna(how='all', subset='Tm')  #.query('Year.str.isnumeric()')
tdf.insert(loc=tdf.columns.get_loc('Salary') + 1, column='Confirmed', value=tdf['Salary'].str[-1].map({'*': False}).astype(bool))
tdf['Tm'] = tdf['Tm'].str.replace('\xa0', ' ')
tdf['Salary'] = tdf['Salary'].str.lstrip('$').str.rstrip('*').str.replace(',', '')
tdf  #.loc[tdf['Year'].isna(), 'Tm'].max()

,Year,Age,Tm,Salary,Confirmed,SrvTm,Sources,Notes/Other Sources
0,2017,23,Atlanta Braves,540000,True,0.047,contracts,NaN
1,2018,24,Atlanta Braves,565000,True,1.045,NaN,NaN
2,2019,25,Atlanta Braves,585000,True,2.047,contract,NaN
3,2020,26,Atlanta Braves,3150000,True,3.047,contract,NaN
4,2021,27,Atlanta Braves,6000000,True,4.047,contract,NaN
5,2022,28,Atlanta Braves,10000000,True,5.047,contract,NaN
6,2023,29,Chicago Cubs,14000000,True,6.047,NaN,NaN
8,2024,30,Chicago Cubs,26000000,True,7.047,NaN,NaN
9,2025,31,Chicago Cubs,28000000,True,NaN,NaN,NaN
10,2026,32,Chicago Cubs,28000000,True,NaN,NaN,NaN


In [97]:
geolocator = Nominatim(user_agent='google')
geocode = partial(geolocator.geocode, exactly_one=True, addressdetails=True, language='en', timeout=10)
suffixes = ['Jr.', 'Sr.', 'II', 'III', 'IV', 'V']

nested_dict = lambda: defaultdict(nested_dict)
test_dict = nested_dict()
player_id = 'younger01'
url = f'https://www.baseball-reference.com/players/{player_id[0]}/{player_id}.shtml'

soup = BeautifulSoup(requests.get(url).content, 'html.parser', from_encoding='utf_8')
header = json.loads(soup.find(type='application/ld+json').string)
name = header['name'].strip().split()

if len(name) == 1:
    test_dict['name']['roster_last'] = name[0]
elif len(name) == 2:
    test_dict['name']['roster_first'], test_dict['name']['roster_last'] = name
elif len(name) == 3 and name[-1] in suffixes:
    test_dict['name']['roster_first'], test_dict['name']['roster_last'] = name[0], ' '.join(name[1:])
else:
    try:
        idx = len(name) - [name.startswith(''.join([char for char in player_id if char.isalpha()])[:-2]) for name in [unidecode(name.lower()) for name in name]][::-1].index(True) - 1
    except ValueError:
        m = [char for char in player_id if char.isalpha()][:-2]
        idx = (np.cumsum([len(n) for n in name]) > re.search(''.join([e for tup in list(zip(m, [r'\W*'] * len(m))) for e in tup]), ''.join([unidecode(name.lower()) for name in name])).start()).argmax()
    test_dict['name']['roster_first'], test_dict['name']['roster_last'] = ' '.join(name[:idx]), ' '.join(name[idx:])
    
name_full = soup.find('strong', string='Full Name:').next_sibling.strip().split()

if name_full[-1] in suffixes:
    name_full, name_suffix = name_full[:-1], name_full[-1]
else:
    name_suffix = ''

if ' '.join(name_full) == ' '.join([n for n in name if n not in suffixes]):
    test_dict['name']['first'], test_dict['name']['last'] = test_dict['name']['roster_first'], test_dict['name']['roster_last']
    if name_suffix and name_suffix not in test_dict['name']['last']:
        test_dict['name']['last'] += ' ' + name_suffix
elif len(name_full) == 2:
    test_dict['name']['first'], test_dict['name']['last'] = name_full[0], name_full[-1] + ' ' + name_suffix
else:
    if ' '.join(name_full).endswith(re.sub('( ' + '| '.join(suffixes).replace('.', r'\.') + ')$', '', test_dict['name']['roster_last'])):
        print('True')
        last = test_dict['name']['roster_last']
        if any(last.endswith(s) for s in suffixes) and not name_suffix:
            last = re.sub('( ' + '| '.join(suffixes).replace('.', r'\.') + ')$', '', last)
        elif name_suffix and not last.endswith(name_suffix):
            last += ' ' + name_suffix
        name_full.reverse()
        for n in test_dict['name']['roster_last'].split():
            if n not in suffixes:
                name_full.remove(n)
        name_full.reverse()
    else:
        last = name_full.pop(-1) + ' ' + name_suffix
    test_dict['name']['first'] = name_full.pop(0)
    if name_full:
        # print(name_full)
        middle = ' '.join(name_full)
        if not re.match(r'[A-Z]\.', middle):
            test_dict['name']['middle'] = middle
        test_dict['name']['middle_init'] = middle[0]
    test_dict['name']['last'] = last

test_dict

True


defaultdict(<function __main__.<lambda>()>,
            {'name': defaultdict(<function __main__.<lambda>()>,
                         {'roster_first': 'Eric',
                          'roster_last': 'Young Sr.',
                          'first': 'Eric',
                          'middle': 'Orlando',
                          'middle_init': 'O',
                          'last': 'Young'})})

In [92]:
idx

2

In [161]:
datetime.strptime('August 26, 2022', '%B %d, %Y').year

ValueError: time data 'January, 2022' does not match format '%B %d, %Y'

In [107]:
drafts = re.split(', the | and the ', ' '.join(soup.find('strong', string='Draft').parent.text.split()).replace('Draft: Drafted by the ', '')[:-1])
drafts

['Kansas City Athletics in the 1st round (1st) of the 1966 MLB August Legion Draft',
 'Houston Astros in the 1st round (1st) of the 1967 MLB January Draft-Secondary Phase from Deerfield Academy (Deerfield, MA)',
 'Houston Astros in the 3rd round of the 1967 MLB June Draft-Secondary Phase from Deerfield Academy (Deerfield, MA)',
 'Washington Senators in the 2nd round of the 1969 MLB June Draft-Secondary Phase from Harvard University (Cambridge, MA)',
 'San Francisco Giants in the 2nd round of the 1970 MLB June Draft-Secondary Phase from Harvard University (Cambridge, MA)',
 'Atlanta Braves in the 1st round (20th) of the 1971 MLB January Draft-Secondary Phase from Harvard University (Cambridge, MA)',
 'Chicago White Sox in the 1st round (1st) of the 1971 MLB June Draft-Secondary Phase from Harvard University (Cambridge, MA)']

In [132]:
geolocator = Nominatim(user_agent='google')
geocode = partial(geolocator.geocode, exactly_one=True, addressdetails=True, language='en', timeout=10)
raw_drafts = re.split(', the | and the ', ' '.join(soup.find('strong', string='Draft').parent.text.split()).replace('Draft: Drafted by the ', '')[:-1])
drafts = []
for draft in raw_drafts:
    m = re.match(r'(?P<team>.+) in the (?P<round>\d+)[a-z]{2} round(?: \((?P<pick>\d+)[a-z]{2}\))? of the (?P<year>\d{4}) MLB (?P<month>January|June|August).* Draft(?:-(?P<secondary_phase>Secondary Phase))?(?: from (?P<school>.+) \((?P<city>.+), (?P<state>[A-Z]{2})\))?', draft).groupdict()
    m['month'] = list(calendar.month_name).index(m['month'])
    m['secondary_phase'] = 'Secondary Phase' in draft
    if m['school']:
        try:
            raw_address = geocode(f"{m['city']}, {m['state']}").raw['address']
        except AttributeError:
            try:
                raw_address = geocode(m['school']).raw['address']
            except AttributeError:
                raw_address = geocode(STATE_DICT[m['state']]).raw['address']
                raw_address['city'] = m['city']
        
        m['school'] = {'name': m['school']}
        match_location(loc_dict=m['school'], raw_address=raw_address)
        del m['city'], m['state']
            
    drafts.append({key: int(value) if key in {'round', 'pick', 'year'} else value for key, value in m.items() if key == 'secondary_phase' or value})

drafts

[{'team': 'Kansas City Athletics',
  'round': 1,
  'pick': 1,
  'year': 1966,
  'month': 8,
  'secondary_phase': False},
 {'team': 'Houston Astros',
  'round': 1,
  'pick': 1,
  'year': 1967,
  'month': 1,
  'secondary_phase': True,
  'school': {'name': 'Deerfield Academy',
   'city': 'Deerfield',
   'state': 'Massachusetts',
   'country': 'United States'}},
 {'team': 'Houston Astros',
  'round': 3,
  'year': 1967,
  'month': 6,
  'secondary_phase': True,
  'school': {'name': 'Deerfield Academy',
   'city': 'Deerfield',
   'state': 'Massachusetts',
   'country': 'United States'}},
 {'team': 'Washington Senators',
  'round': 2,
  'year': 1969,
  'month': 6,
  'secondary_phase': True,
  'school': {'name': 'Harvard University',
   'city': 'Cambridge',
   'state': 'Massachusetts',
   'country': 'United States'}},
 {'team': 'San Francisco Giants',
  'round': 2,
  'year': 1970,
  'month': 6,
  'secondary_phase': True,
  'school': {'name': 'Harvard University',
   'city': 'Cambridge',
   'sta

In [0]:
player_id = 'swansda01'
url = f'https://www.baseball-reference.com/players/{player_id[0]}/{player_id}.shtml'

geolocator = Nominatim(user_agent='google')
geocode = partial(geolocator.geocode, exactly_one=True, addressdetails=True, language='en')
soup = BeautifulSoup(requests.get(url).content, 'html.parser', from_encoding='utf_8')

nested_dict = lambda: defaultdict(nested_dict)
test_dict = nested_dict()

header = json.loads(soup.find(type='application/ld+json').string)
name = header['name'].strip().split()
if len(name) == 1:
    test_dict['name']['last'] = name[0]
elif len(name) == 2:
    test_dict['name']['first'], test_dict['name']['last'] = name
elif len(name) == 3 and name[-1] in ['Jr.', 'Sr.', 'II', 'III', 'IV', 'V']:
    test_dict['name']['first'], test_dict['name']['last'] = name[0], ' '.join(name[1:])
else:
    idx = len(name) - [name.startswith(''.join([char for char in player_id if char.isalpha()])[:-2]) for name in [unidecode(name.lower()) for name in name]][::-1].index(True) - 1
    test_dict['name']['first'], test_dict['name']['last'] = ' '.join(name[:idx]), ' '.join(name[idx:])

try:
    birth_date = [int(d) for d in header['birthDate'].split('-')]
    for i, date_part in enumerate(['year', 'month', 'day']):
        if birth_date[i] > 0:
            test_dict['born'][date_part] = birth_date[i]
except KeyError:
    pass

try:
    match_location(location=header['birthPlace'], loc_dict=test_dict['born'], geocode=geocode)
except KeyError:
    pass

try:
    test_dict['height']['inches'] = (np.array(header['height']['value'].split('-'), dtype=int) * np.array([12, 1])).sum()
    test_dict['height']['feet'] = test_dict['height']['inches'] / 12.0
    test_dict['height']['meters'] = test_dict['height']['feet'] * 0.3048
except KeyError:
    pass

try:
    test_dict['weight']['lbs'] = int(header['weight']['value'].replace(' lbs', ''))
    test_dict['weight']['kg'] = test_dict['weight']['lbs'] * 0.453592
except KeyError:
    pass

with soup.find('strong', string='Bats: ').next_sibling.strip()[0] as bats:
    if bats != 'U':
        test_dict['bats'] = bats

with soup.find('strong', string='Bats: ').next_sibling.strip()[0] as throws:
    if throws != 'U':
        test_dict['bats'] = bats

try:
    death_date = [int(d) for d in soup.find('span', id='necro-death')['data-death'].split('-')]
    for i, date_part in enumerate(['year', 'month', 'day']):
        if death_date[i] > 0:
            test_dict['died'][date_part] = death_date[i]
except KeyError:
    pass

try:
    match_location(location=soup.find('span', id='necro-death').find_next_sibling('span').text.replace('in', '').replace('\xa0', ' ').strip(), loc_dict=test_dict['died'], geocode=geocode)
except AttributeError:
    pass

if player_id == 'kleinnu01':
    test_dict['died']['cemetery'] = 'Atlantic Ocean'
else:
    try:
        location = list(soup.find('a', string='Buried').next_elements)[2].strip()
        if location[0] != 'Unknown Cemetery':
            test_dict['died']['cemetery'] = location[0]
        if location != 'Cremated':
            try:
                match_location(location=', '.join(location.split(', ')[1:]), loc_dict=test_dict['died']['buried'], geocode=geocode)
            except AttributeError:
                pass
    except AttributeError:
        pass
try:
    raw_drafts = re.split(', the | and the ', ' '.join(soup.find('strong', string='Draft').parent.text.split()).replace('Draft: Drafted by the ', '')[:-1])
    drafts = []
    for draft in raw_drafts:
        m = re.match(r'(?P<team>.+) in the (?P<round>\d+)[a-z]{2} round(?: \((?P<pick>\d+)[a-z]{2}\))? of the (?P<year>\d{4}) MLB (?P<month>January|June|August).* Draft(?:-(?P<secondary_phase>Secondary Phase))?(?: from (?P<school>.+) \((?P<city>.+), (?P<state>[A-Z]{2})\))?', draft).groupdict()
        m['month'] = list(calendar.month_name).index(m['month'])
        m['secondary_phase'] = 'Secondary Phase' in draft
        if m['school']:
            try:
                raw_address = geocode(f"{m['city']}, {m['state']}").raw['address']
            except AttributeError:
                try:
                    raw_address = geocode(m['school']).raw['address']
                except AttributeError:
                    raw_address = geocode(STATE_DICT[m['state']]).raw['address']
                    raw_address['city'] = m['city']
    
            m['school'] = {'name': m['school']}
            match_location(loc_dict=m['school'], raw_address=raw_address)
            del m['city'], m['state']
    
        drafts.append({key: int(value) if key in {'round', 'pick', 'year'} else value for key, value in m.items() if key == 'secondary_phase' or value})
    
    test_dict['drafts'] = drafts
except AttributeError:
    pass

for school_type in ['high_schools', 'colleges']:
    try:
        schools = [dict(zip(['name', 'city', 'state'], re.split(r' \(|, (?=[A-Z]{2})', school.rstrip(')')))) for school in re.split(r'(?<=\)), ', soup.find('strong', string=re.compile(('High ' if school_type == 'high_schools' else '') + 'School(s)?:')).find_next_sibling('a').text)]
        for school in schools:
            try:
                raw_address = geocode(f"{school['city']}, {school['state']}").raw['address']
            except AttributeError:
                try:
                    raw_address = geocode(school['school']).raw['address']
                except AttributeError:
                    raw_address = geocode(STATE_DICT[school['state']]).raw['address']
                    raw_address['city'] = school['city']
        
            match_location(school, raw_address=raw_address)
        
        test_dict[school_type] = schools
    except AttributeError:
        pass

for event in ['debut', 'finale']:
    try:
        debut_info = [info.string.strip().replace(',', '').replace('vs. ', '').split() for info in soup.find('strong', string='Debut:' if event == 'debut' else 'Last Game:').find_next_siblings('a')]
        debut = [int(d) if d.isnumeric() else list(calendar.month_name).index(d) for d in debut_info[0]]
        test_dict[event]['year'] = debut[-1]
        if len(debut) > 1:
            test_dict[event]['month'] = debut[0]
            if len(debut) == 3:
                test_dict[event]['day'] = debut[1]
        if len(debut_info) > 1:
            test_dict[event]['opponent'] = debut_info[1]
            
    except AttributeError:
        pass

try:
    test_dict['last_rookie_season'] = int(re.search(r'\d{4}', soup.find('strong', string='Rookie Status:').next_sibling.strip())[0])
except AttributeError:
    pass

try:
    test_dict['agents'] = [{'name': agent, 'current': len(agents) == 1} for agents in [a.replace('\u2022', '').replace(':', '').strip().split(', ') for a in soup.find('strong', string='Agents').next_siblings if isinstance(a, str)] for agent in agents]
except AttributeError:
    pass

In [4]:

tdf = soup.find('table')
tdf, table_id = pd.read_html(StringIO(str(soup.find('table'))))[0].dropna(subset='Tm').query("Year.str.isnumeric() & ~Tm.str.match('.+-min|TOT')"), tdf['id']
tdf = tdf.astype({col: 'Int64' for col in INT_COLS & set(tdf.columns)} | {col: float for col in FLOAT_COLS & set(tdf.columns)})
if table_id == 'pitching_standard':
    tdf['IP'] = tdf['IP'].mul(3).round().div(3)

tdf.insert(loc=0, column='PlayerID', value=player_id)
test_dict[table_id] = tdf

try:
    table_id = 'batting_standard' if table_id == 'pitching_standard' else 'pitching_standard'
    tdf = pd.read_html(StringIO(str(BeautifulSoup([comment.strip() for comment in soup.find_all(string=lambda text: isinstance(text, Comment)) if 'table' in comment and bool(re.search(f'id="{table_id}"', comment))][0]).find('table'))))[0].dropna(subset='Tm').query("Year.str.isnumeric() & ~Tm.str.match('.+-min|TOT')")
    tdf = tdf.astype({col: 'Int64' for col in INT_COLS & set(tdf.columns)} | {col: float for col in FLOAT_COLS & set(tdf.columns)})
    if table_id == 'pitching_standard':
        tdf['IP'] = tdf['IP'].mul(3).round().div(3)
        
    tdf.insert(loc=0, column='PlayerID', value=player_id)
    test_dict[table_id] = tdf
except IndexError:
    pass

for player_stats in [ps.replace('_standard', '') for ps in test_dict]:
    for suffix in ['_value', '_advanced', '_postseason']:
        try:
            table_id = player_stats + suffix
            tdf = pd.read_html(StringIO(str(BeautifulSoup([comment.strip() for comment in soup.find_all(string=lambda text: isinstance(text, Comment)) if 'table' in comment and bool(re.search(f'id="{table_id}"', comment))][0]).find('table'))))[0]
            if suffix == '_advanced':
                tdf = tdf.droplevel(0, axis=1)
            elif suffix == '_value':
                tdf['Salary'] = tdf['Salary'].str.lstrip('$').str.replace(',', '').astype(float)
                
            tdf = tdf.dropna(subset='Tm').query("Year.str.isnumeric() & ~Tm.str.match('TOT')")
            tdf = tdf.astype({col: 'Int64' for col in INT_COLS & set(tdf.columns)} | {col: float for col in FLOAT_COLS & set(tdf.columns)})
            tdf[list({'HR%', 'SO%', 'BB%', 'LD%', 'GB%', 'FB%', 'Pull%', 'Cent%', 'Oppo%', 'cWPA', 'RS%', 'SB%', 'XBT%'} & set(tdf.columns))] = tdf[list({'HR%', 'SO%', 'BB%', 'LD%', 'GB%', 'FB%', 'Pull%', 'Cent%', 'Oppo%', 'cWPA', 'RS%', 'SB%', 'XBT%'} & set(tdf.columns))].apply(lambda x: x.astype(str).str.rstrip('%').astype(float).div(100.0))
            if player_stats == 'pitching':
                tdf['IP'] = tdf['IP'].mul(3).round().div(3)
            
            tdf.insert(loc=0, column='PlayerID', value=player_id)
            test_dict[table_id] = tdf
        except IndexError:
            continue

try:
    tdf = pd.read_html(StringIO(str(BeautifulSoup([comment.strip() for comment in soup.find_all(string=lambda text: isinstance(text, Comment)) if 'table' in comment and bool(re.search('id="standard_fielding"', comment))][0]).find('table'))))[0].rename({'PO.1': 'PK'}, axis=1).dropna(subset='Tm').query("Year.str.isnumeric() & ~Tm.str.match('TOT')")
    tdf = tdf.astype({col: 'Int64' for col in INT_COLS & set(tdf.columns)} | {col: float for col in FLOAT_COLS & set(tdf.columns)})
    try:
        tdf[['CS%', 'lgCS%']] = tdf[['CS%', 'lgCS%']].apply(lambda x: x.astype(str).str.rstrip('%').astype(float).div(100.0))
    except KeyError:
        pass

    tdf.insert(loc=0, column='PlayerID', value=player_id)
    test_dict['fielding_standard'] = tdf
except IndexError:
    pass

table_id = 'appearances'
tdf = pd.read_html(StringIO(str(BeautifulSoup([comment.strip() for comment in soup.find_all(string=lambda text: isinstance(text, Comment)) if 'table' in comment and bool(re.search(f'id="{table_id}"', comment))][0]).find('table'))))[0].dropna(subset='Tm').query("Year.str.isnumeric() & ~Tm.str.match('.+-minTOT')").astype({col: int for col in {'Year', 'Age'}})

tdf.insert(loc=0, column='PlayerID', value=player_id)
test_dict[table_id] = tdf

del tdf

In [5]:
test_dict.keys()

dict_keys(['batting_standard', 'batting_value', 'batting_advanced', 'batting_postseason', 'fielding_standard', 'appearances'])

In [ ]:
for letter in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']:
    players = {pid for pid in set(pd.read_csv('bref_standard_batting.csv')['PlayerID'].unique()) | set(pd.read_csv('bref_standard_pitching.csv')['PlayerID'].unique()) | set(pd.read_csv('bref_standard_fielding.csv')['PlayerID'].unique()) if pid.startswith(letter)}
    player_dict = defaultdict(dict)
    
    pbar = tqdm(players, desc='Scraping Baseball-Reference player pages', unit='players', dynamic_ncols=True, leave=False)
    
    for player in pbar:
        pbar.set_postfix({'playerIDBR': player})
        url = f'https://www.baseball-reference.com/players/{player[0]}/{player}.shtml'
        tdf = scrape_table(url, extract_links=False).dropna(how='all', subset='Tm').query("Year.str.isnumeric() & ~Tm.str.match('.+-min|TOT')")
        tdf = tdf.astype({col: 'Int64' for col in INT_COLS & set(tdf.columns)} | {col: float for col in FLOAT_COLS & set(tdf.columns)})
        tdf.insert(loc=0, column='PlayerID', value=player)

        if 'OPS+' in tdf.columns:
            batting = True
            player_dict[player]['standard_batting'] = tdf
        elif 'ERA+' in tdf.columns:
            batting = False
            tdf['IP'] = tdf['IP'].mul(3).round().div(3)
            player_dict[player]['standard_pitching'] = tdf
        else:
            tdf['Inn'] = tdf['Inn'].mul(3).round().div(3)
            if 'SB' in tdf.columns:
                tdf = tdf.rename({'PO.1': 'PK'}, axis=1)
                tdf[['CS%', 'lgCS%']] = tdf[['CS%', 'lgCS%']].apply(lambda x: x.astype(str).str.rstrip('%').astype(float).div(100.0))
            player_dict[player]['standard_fielding'] = tdf
            continue

        try:
            tdf = scrape_table(url, match='Standard Pitching' if batting else 'Standard Batting', extract_links=False, commented=True).dropna(how='all', subset='Tm').query("Year.str.isnumeric() & ~Tm.str.match('.+-min|TOT')")
            tdf = tdf.astype({col: 'Int64' for col in INT_COLS & set(tdf.columns)} | {col: float for col in FLOAT_COLS & set(tdf.columns)})
            if not batting:
                tdf['IP'] = tdf['IP'].mul(3).round().div(3)
            tdf.insert(loc=0, column='PlayerID', value=player)
            player_dict[player]['standard_pitching' if batting else 'standard_pitching'] = tdf
        except IndexError:
            player_dict[player]['standard_pitching' if batting else 'standard_pitching'] = pd.DataFrame()
    
        try:
            tdf = scrape_table(url, match='Standard Fielding', extract_links=False, commented=True).dropna(how='all', subset='Tm').query("Year.str.isnumeric() & ~Tm.str.match('.+-min|TOT')")
            tdf = tdf.astype({col: 'Int64' for col in INT_COLS & set(tdf.columns)} | {col: float for col in FLOAT_COLS & set(tdf.columns)})
            tdf['Inn'] = tdf['Inn'].mul(3).round().div(3)
            if 'SB' in tdf.columns:
                tdf = tdf.rename({'PO.1': 'PK'}, axis=1)
                tdf[['CS%', 'lgCS%']] = tdf[['CS%', 'lgCS%']].apply(lambda x: x.astype(str).str.rstrip('%').astype(float).div(100.0))
            tdf.insert(loc=0, column='PlayerID', value=player)
            player_dict[player]['standard_fielding'] = tdf
        except IndexError:
            player_dict[player]['standard_fielding'] = pd.DataFrame()
        sleep(3)
    
    pd.concat([value['standard_fielding'] for value in player_dict.values()], ignore_index=True).to_csv(f'bref_standard_fielding_{letter}.csv', index=False, chunksize=50000)

Scraping Baseball-Reference player pages:   0%|          | 0/651 [00:00<?, ?players/s]

In [29]:
pd.concat([value['standard_fielding'] for value in player_dict.values()], ignore_index=True).to_csv('bref_standard_fielding_a.csv', index=False, chunksize=50000)

,PlayerID,Year,Age,Tm,Pos,Lg,G,GS,CG,Inn,...,lgFld%,lgRF9,lgRFG,SB,CS,CS%,lgCS%,PK,Awards,LF-CF-RF
0,amarial01,2011,22,LAA,2B,AL,14,9,7,93.333333,...,.984,4.77,4.73,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN
1,amarial01,2011,22,LAA,LF,AL,8,4,2,43.000000,...,.982,2.12,2.11,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN
2,amarial01,2011,22,LAA,OF,AL,8,4,2,43.000000,...,.986,2.28,2.26,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN
3,amarial01,2011,22,LAA,SS,AL,1,0,0,3.000000,...,NaN,4.50,4.30,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN
4,amarial01,2012,23,LAA,DH,AL,1,0,<NA>,NaN,...,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,NaN,2016,25,TBR,RF,AL,11,10,8,83.000000,...,.986,2.10,2.08,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN
22,NaN,2016,25,TBR,DH,AL,3,3,<NA>,NaN,...,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN
23,NaN,2016,25,TBR,LF,AL,3,3,2,24.000000,...,.983,1.88,1.83,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN
24,NaN,2016,25,SDP,OF,NL,13,11,10,97.333333,...,.985,2.07,2.05,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN


In [12]:
tdf = scrape_table('https://www.baseball-reference.com/players/a/alvarvi01.shtml', match='Standard Fielding', extract_links=False, commented=True).dropna(how='all', subset='Tm').query("Year.str.isnumeric() & ~Tm.str.match('.+-min|TOT')")
tdf = tdf.astype({col: 'Int64' for col in int_cols & set(tdf.columns)} | {col: float for col in float_cols & set(tdf.columns)})
tdf['Inn'] = tdf['Inn'].mul(3).round().div(3)
tdf

,Year,Age,Tm,Pos,Lg,G,GS,CG,Inn,Ch,...,RF/G,lgFld%,lgRF9,lgRFG,SB,CS,CS%,lgCS%,PO.1,Awards
0,2002,25,LAD,P,NL,4,1,0,10.000000,1,...,0.25,0.96,1.83,1.80,0,0,NaN,NaN,0,NaN
1,2003,26,LAD,P,NL,5,0,0,5.333333,0,...,0.00,NaN,1.91,1.84,0,0,NaN,NaN,0,NaN


In [20]:
tdf[['CS%', 'lgCS%']].apply(lambda x: x.astype(str).str.rstrip('%').astype(float).div(100.0))

,CS%,lgCS%
0,NaN,NaN
1,NaN,NaN


In [ ]:
tdf = scrape_table(page, match='Standard Pitching', extract_links=False, commented=True).dropna(how='all', subset=['Year', 'Age', 'Tm']).query("Year.str.isnumeric() & ~Tm.str.match('.+-min|TOT')")
tdf.astype({col: 'Int64' for col in tdf.columns if col in int_cols} | {col: float for col in tdf.columns if col in float_cols}, errors='ignore')

In [111]:
tdf = scrape_table(page, match='Standard Fielding', extract_links=False, commented=True).dropna(how='all', subset=['Year', 'Age', 'Tm']).query("Year.str.isnumeric() & ~Tm.str.match('.+-min|TOT')")
tdf.astype({col: 'Int64' for col in tdf.columns if col in int_cols} | {col: float for col in tdf.columns if col in float_cols}, errors='ignore')

,Year,Age,Tm,Pos,Lg,G,GS,CG,Inn,Ch,...,RF/G,lgFld%,lgRF9,lgRFG,SB,CS,CS%,lgCS%,PO.1,Awards
0,2011,21,SDP,1B,NL,45,37,34,341.2,344,...,7.60,.994,9.33,9.31,<NA>,<NA>,NaN,NaN,NaN,NaN
1,2012,22,CHC,1B,NL,85,85,82,730.2,735,...,8.60,.992,9.29,9.18,<NA>,<NA>,NaN,NaN,NaN,NaN
2,2013,23,CHC,1B,NL,159,158,158,1415.0,1441,...,9.03,.994,9.42,9.41,<NA>,<NA>,NaN,NaN,NaN,NaN
3,2014,24,CHC,1B,NL,140,140,135,1259.0,1311,...,9.30,.993,9.29,9.27,<NA>,<NA>,NaN,NaN,NaN,"AS,MVP-10"
4,2015,25,CHC,1B,NL,160,160,154,1430.1,1465,...,9.10,.994,9.30,9.23,<NA>,<NA>,NaN,NaN,NaN,"AS,MVP-4"
5,2016,26,CHC,1B,NL,154,151,140,1337.0,1399,...,9.05,.994,9.05,8.99,<NA>,<NA>,NaN,NaN,NaN,"AS,MVP-4,GG,SS"
6,2016,26,CHC,2B,NL,1,0,0,0.1,1,...,1.00,.980,5.40,4.60,<NA>,<NA>,NaN,NaN,NaN,"AS,MVP-4,GG,SS"
7,2017,27,CHC,1B,NL,157,155,130,1341.0,1375,...,8.74,.994,8.94,8.84,<NA>,<NA>,NaN,NaN,NaN,MVP-13
8,2017,27,CHC,2B,NL,10,0,0,4.1,2,...,0.20,.980,4.36,4.36,<NA>,<NA>,NaN,NaN,NaN,MVP-13
9,2017,27,CHC,3B,NL,1,0,0,1.0,0,...,0.00,NaN,2.70,2.50,<NA>,<NA>,NaN,NaN,NaN,MVP-13


In [70]:
[comment.strip() for comment in BeautifulSoup(requests.get(page).content, 'html.parser', from_encoding='utf_8').find_all(string=lambda text: isinstance(text, Comment)) if 'table' in comment and 'Standard Pitching' in comment][0]

'<div class="table_container" id="div_pitching_standard">\n    \n    <table class="row_summable sortable stats_table" id="pitching_standard" data-cols-to-freeze="1,3">\n    <caption>Standard Pitching</caption>\n    \n\n   <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup>\n   <thead>      \n      <tr>\n         <th aria-label="Year" data-stat="year_ID" scope="col" class=" poptip sort_default_asc show_partial_when_sorting left" data-tip="A Star indicates an all-star that season.&lt;br&gt;A Ring indicates the player appeared in WS for winning team." >Year</th>\n         <th aria-label="Age" data-stat="age" scope="col" class=" poptip sort_default_asc show_partial_when_sorting center" data-tip="Player&#x2019;s age at midnight of June 30th of that year" >Age</th>\n         <th aria-label="Tm" data-stat="team_ID" scope="col" class=" poptip sort_def

In [23]:
if SCRAPE:
    df = fangraphs_team_scrape()
    df.to_csv('team_batting.csv', index=False)
else:
    df = pd.read_csv('team_batting.csv')[['Season', 'TeamName', 'teamid']].rename({'Season': 'yearID', 'TeamName': 'teamAbrFG', 'teamid': 'franchIDFG'}, axis=1)
    teams = team_ids()[['yearID', 'teamIDfg', 'franchID', 'teamIDBR']].rename({'franchID': 'franchIDLahman', 'teamIDfg': 'franchIDFG', 'teamIDBR': 'franchIDBR'}, axis=1)
    teams = pd.concat([teams, teams.loc[teams['yearID'] == 2021, ['franchIDFG', 'franchIDLahman', 'franchIDBR']].assign(yearID=2022), teams.loc[teams['yearID'] == 2021, ['franchIDFG', 'franchIDLahman', 'franchIDBR']].assign(yearID=2023)])  # duplicate 2021 data for 2022 and 2023
    
    df = df.merge(teams, how='left', on=['yearID', 'franchIDFG']).replace(LAHMAN_CORRECTIONS).merge(FG_TO_BR_NA_TEAM_MAPPING, how='left', left_on='franchIDFG', right_index=True)
    df['franchIDLahman'] = df['franchIDLahman'].fillna(df['franchIDFG'].map(FG_TO_LAHMAN_FRANCH_MAPPING))
    df['franchIDBR'] = df['franchIDBR'].fillna(df['franchIDFG'].map(FG_TO_BR_FRANCH_MAPPING))
    df['franchIDBR'] = df['franchIDBR'].where(df['yearID'] > 1877, df['franchIDBR'].replace(BR_FRANCH_CORRECTIONS))
    df['teamAbrBR'] = df['teamAbrBR'].fillna(df['franchIDBR'])
    
    df.loc[(df['yearID'] == 1880) & (df['franchIDFG'] == 1028), ['franchIDFG', 'franchIDLahman', 'franchIDBR']] = [1091, 'CNS', 'CNS']
    df.loc[df['yearID'].isin([1901, 1902]) & (df['franchIDFG'] == 9), ['franchIDFG', 'franchIDLahman', 'franchIDBR']] = [1092, 'BLA', 'BLA']
    franch_ids = df.loc[df['franchIDFG'].isin(df.groupby('franchIDFG')['franchIDBR'].nunique().sort_values(ascending=False).index.tolist()[:18]), ['yearID', 'franchIDFG', 'franchIDBR']].sort_values(['franchIDFG', 'yearID']).drop_duplicates(subset='franchIDFG', keep='last', ignore_index=True)[['franchIDFG', 'franchIDBR']]
    
    df = df.merge(franch_ids, how='left', on='franchIDFG', suffixes=('_p', ''))
    df['franchIDBR'] = df['franchIDBR'].fillna(df['franchIDBR_p'])
    df = df.drop('franchIDBR_p', axis=1)
    
    df.loc[df['franchIDFG'].isin(BR_TO_LAHMAN), 'franchIDBR'] = df.loc[df['franchIDFG'].isin(BR_TO_LAHMAN), 'franchIDLahman']
    
franchises = df[['franchIDFG', 'franchIDBR', 'franchIDLahman', 'yearID']].drop_duplicates().sort_values(['franchIDFG', 'yearID'], ignore_index=True)
f = franchises.groupby('franchIDFG')['yearID'].agg(['min', 'max', 'size']).rename({'min': 'yearFirst', 'max': 'yearLast', 'size': 'yearsActive'}, axis=1)

franchises = franchises[['franchIDFG', 'franchIDBR', 'franchIDLahman']].drop_duplicates(ignore_index=True).merge(f, left_on='franchIDFG', right_index=True).merge(pd.concat([pd.read_csv('baseballdatabank-2022.2/core/TeamsFranchises.csv')[['franchID', 'franchName', 'active']].rename({'active': 'isActive'}, axis=1), pd.DataFrame(data=[['CNS', 'Cincinnati Stars', 'N'], ['BLA', 'Baltimore Orioles', 'N']], columns=['franchID', 'franchName', 'isActive'])], ignore_index=True).fillna('N'), left_on='franchIDLahman', right_on='franchID').drop('franchID', axis=1).sort_values(['isActive', 'yearFirst'], ascending=[False, True]).replace({'franchName': {'Cleveland Indians': 'Cleveland Guardians', 'Los Angeles Angels of Anaheim': 'Los Angeles Angels', 'Florida Marlins': 'Miami Marlins', 'Altoona Mountain City': 'Altoona Mountain Citys'}})
franchises['isActive'] = franchises['isActive'].map({'Y': True, 'N': False})

franchises.index = pd.Index(data=range(1, franchises.shape[0] + 1), name='franchID')

In [24]:
df = franchises[['franchIDFG', 'franchIDBR', 'franchIDLahman']].reset_index().merge(df, on=['franchIDFG', 'franchIDBR', 'franchIDLahman'])

In [25]:
lahman = pd.read_csv('baseballdatabank-2022.2/core/Teams.csv')[['yearID', 'lgID', 'divID', 'teamID', 'teamIDBR', 'teamIDlahman45', 'teamIDretro', 'franchID', 'name']].fillna({'lgID': 'NA'}).rename({'teamID': 'teamAbrLahman', 'teamIDBR': 'teamAbrBR', 'teamIDlahman45': 'teamAbrLahman45', 'franchID': 'franchIDLahman', 'teamIDretro': 'teamAbrRetro', 'name': 'teamName'}, axis=1).replace({'franchIDLahman': LAHMAN_CORRECTIONS, 'teamAbrRetro': {'BL5': 'BL2'}, 'teamName': {'Philadelphia Whites': 'Philadelphia White Stockings', 'Chicago/Pittsburgh (Union League)': 'Chicago Browns/Pittsburgh Stogies', 'Pittsburg Alleghenys': 'Pittsburgh Alleghenys', 'Philadelphia Quakers': 'Philadelphia Phillies', 'Buffalo Buffeds': 'Buffalo Blues', 'Chicago Chi-Feds': 'Chicago Whales', 'St. Paul White Caps': 'St. Paul Saints', 'Newark Pepper': 'Newark Peppers', 'Philadelphia Blue Jays': 'Philadelphia Phillies', "Houston Colt .45's": 'Houston Colt .45s'}})

lahman = pd.concat([lahman, lahman[lahman['yearID'] == 2021].assign(yearID=2022), lahman[lahman['yearID'] == 2021].assign(yearID=2023)])

lahman.loc[(lahman['yearID'] == 1880) & (lahman['teamName'] == 'Cincinnati Reds'), 'teamName'] = 'Cincinnati Stars'
lahman.loc[(lahman['yearID'] >= 2016) & (lahman['teamName'] == 'Los Angeles Angels of Anaheim'), 'teamName'] = 'Los Angeles Angels'
lahman.loc[(lahman['yearID'] >= 2022) & (lahman['teamName'] == 'Cleveland Indians'), 'teamName'] = 'Cleveland Guardians'

lahman.loc[(lahman['yearID'] == 1880) & (lahman['franchIDLahman'] == 'CNR'), 'franchIDLahman'] = 'CNS'
lahman.loc[lahman['yearID'].isin([1901, 1902]) & (lahman['franchIDLahman'] == 'NYY'), 'franchIDLahman'] = 'BLA'

home_parks = pd.read_csv('baseballdatabank-2022.2/core/HomeGames.csv')[['year.key', 'team.key', 'park.key', 'games']].rename({'year.key': 'yearID', 'team.key': 'teamAbrLahman', 'park.key': 'parkID'}, axis=1)
home_parks = home_parks[home_parks.groupby(['yearID', 'teamAbrLahman'])['games'].transform('max') == home_parks['games']].drop('games', axis=1)
home_parks = pd.concat([home_parks, home_parks[home_parks['yearID'] == 2021].assign(yearID=2022), home_parks[home_parks['yearID'] == 2021].assign(yearID=2023)])

abr = pd.read_csv('teamabr.csv', names=['teamID', 'leageID', 'city', 'nickname', 'startYearID', 'endYearID']).rename({'teamID': 'teamAbrRetro', 'city': 'location'}, axis=1).replace({'location': {'Ft. Wayne': 'Fort Wayne', 'St.Louis': 'St. Louis'}, 'nickname': {'Forest Cities': 'Forest Citys', 'Cream Citys': 'Grays', 'New Havens': 'Elm Citys', 'Lord Baltimores': 'Canaries', 'Virginias': 'Virginians', 'Mountain Citys': 'Mountain City', 'Wonders': "Ward's Wonders", 'Quakers': 'Athletics', 'Hop Bitters': 'Broncos'}})
abr['yearID'] = abr.apply(lambda x: range(x['startYearID'], x['endYearID'] + 1), axis=1)
abr = abr.drop(['startYearID', 'endYearID', 'leageID'], axis=1).explode('yearID')
abr = abr[(abr['teamAbrRetro'] != 'CN1') | (abr['yearID'] != 1880)]  # Cincinnati Reds did not play in 1880
abr = pd.concat([abr, abr[abr['yearID'] == 2021].assign(yearID=2022), abr[abr['yearID'] == 2021].assign(yearID=2023)])  # duplicate 2021 data for 2022 and 2023

abr.loc[abr['location'] == 'Chicago-Pittsburgh', ['location', 'nickname']] = ['Chicago/Pittsburgh', 'Browns/Stogies']  # fix Stogies ballpark
abr.loc[abr['yearID'].between(1871, 1875) & (abr['nickname'] == 'Braves'), 'nickname'] = 'Red Stockings'
abr.loc[abr['yearID'].between(1876, 1882) & (abr['nickname'] == 'Braves'), 'nickname'] = 'Red Caps'
abr.loc[abr['yearID'].between(1883, 1906) & (abr['nickname'] == 'Braves'), 'nickname'] = 'Beaneaters'
abr.loc[abr['yearID'].between(1907, 1910) & (abr['nickname'] == 'Braves'), 'nickname'] = 'Doves'
abr.loc[(abr['yearID'] == 1911) & (abr['nickname'] == 'Braves'), 'nickname'] = 'Rustlers'
abr.loc[abr['yearID'].between(1936, 1940) & (abr['nickname'] == 'Braves'), 'nickname'] = 'Bees'
abr.loc[abr['yearID'].between(1889, 1891) & (abr['nickname'] == 'Colts'), 'nickname'] = 'Solons'
abr.loc[abr['yearID'].between(1876, 1889) & (abr['nickname'] == 'Cubs'), 'nickname'] = 'White Stockings'
abr.loc[abr['yearID'].between(1890, 1897) & (abr['nickname'] == 'Cubs'), 'nickname'] = 'Colts'
abr.loc[abr['yearID'].between(1898, 1902) & (abr['nickname'] == 'Cubs'), 'nickname'] = 'Orphans'
abr.loc[(abr['yearID'] == 1873) & (abr['nickname'] == 'Nationals'), 'nickname'] = 'Blue Legs'
abr.loc[(abr['yearID'] == 1875) & (abr['nickname'] == 'Olympics'), 'nickname'] = 'Nationals'
abr.loc[abr['yearID'].between(1882, 1889) & (abr['location'] == 'Cincinnati') & (abr['nickname'] == 'Reds'), 'nickname'] = 'Red Stockings'
abr.loc[abr['yearID'].between(1954, 1959) & (abr['nickname'] == 'Reds'), 'nickname'] = 'Redlegs'
abr.loc[abr['yearID'].between(1883, 1884) & (abr['nickname'] == 'Giants'), 'nickname'] = 'Gothams'
abr.loc[(abr['yearID'] == 1884) & (abr['nickname'] == 'Dodgers'), 'nickname'] = 'Atlantics'
abr.loc[abr['yearID'].between(1885, 1887) & (abr['nickname'] == 'Dodgers'), 'nickname'] = 'Grays'
abr.loc[(abr['yearID'].between(1888, 1890) | abr['yearID'].between(1896, 1898)) & (abr['nickname'] == 'Dodgers'), 'nickname'] = 'Bridegrooms'
abr.loc[abr['yearID'].between(1891, 1895) & (abr['nickname'] == 'Dodgers'), 'nickname'] = 'Grooms'
abr.loc[(abr['yearID'].between(1899, 1910) | (abr['yearID'] == 1913)) & (abr['nickname'] == 'Dodgers'), 'nickname'] = 'Superbas'
abr.loc[abr['yearID'].between(1914, 1931) & (abr['nickname'] == 'Dodgers'), 'nickname'] = 'Robins'
abr.loc[abr['yearID'].between(1879, 1888) & (abr['nickname'] == 'Spiders'), 'nickname'] = 'Blues'
abr.loc[abr['yearID'].between(1882, 1884) & (abr['nickname'] == 'Colonels'), 'nickname'] = 'Eclipse'
abr.loc[abr['yearID'].between(1882, 1890) & (abr['location'] == 'Pittsburgh') & (abr['nickname'] == 'Pirates'), 'nickname'] = 'Alleghenys'
abr.loc[abr['yearID'].between(1883, 1884) & (abr['nickname'] == 'Colts'), 'nickname'] = 'Buckeyes'
abr.loc[(abr['yearID'] == 1884) & (abr['location'] == 'Indianapolis') & (abr['nickname'] == 'Blues'), 'nickname'] = 'Hoosiers'
abr.loc[abr['yearID'].between(1886, 1889) & (abr['nickname'] == 'Senators'), 'nickname'] = 'Nationals'
abr.loc[(abr['yearID'] == 1891) & (abr['nickname'] == 'Senators'), 'nickname'] = 'Statesmen'
abr.loc[(abr['yearID'] == 1882) & (abr['nickname'] == 'Cardinals'), 'nickname'] = 'Brown Stockings'
abr.loc[(abr['yearID'] == 1899) & (abr['nickname'] == 'Cardinals'), 'nickname'] = 'Perfectos'
abr.loc[abr['yearID'].between(1883, 1898) & (abr['nickname'] == 'Cardinals'), 'nickname'] = 'Browns'
abr.loc[(abr['yearID'] == 1901) & (abr['nickname'] == 'Indians'), 'nickname'] = 'Blues'
abr.loc[(abr['yearID'] == 1902) & (abr['nickname'] == 'Indians'), 'nickname'] = 'Bronchos'
abr.loc[abr['yearID'].between(1903, 1914) & (abr['nickname'] == 'Indians'), 'nickname'] = 'Naps'
abr.loc[abr['yearID'].between(1901, 1907) & (abr['nickname'] == 'Red Sox'), 'nickname'] = 'Americans'
abr.loc[abr['yearID'].between(1903, 1912) & (abr['nickname'] == 'Yankees'), 'nickname'] = 'Highlanders'
abr.loc[abr['yearID'].between(1962, 1964) & (abr['nickname'] == 'Colts'), 'nickname'] = 'Colt .45s'
abr.loc[(abr['yearID'] >= 1965) & (abr['nickname'] == 'Colts'), 'nickname'] = 'Astros'
abr.loc[(abr['yearID'] >= 2005) & (abr['location'] == 'Anaheim'), 'location'] = 'Los Angeles'
abr.loc[(abr['yearID'] >= 2008) & (abr['nickname'] == 'Devil Rays'), 'nickname'] = 'Rays'
abr.loc[(abr['yearID'] >= 2022) & (abr['nickname'] == 'Indians'), 'nickname'] = 'Guardians'

teams = lahman.merge(abr, on=['yearID', 'teamAbrRetro']).merge(home_parks, how='left', on=['yearID', 'teamAbrLahman']).merge(df[['yearID', 'franchID', 'teamAbrFG', 'franchIDLahman']], on=['yearID', 'franchIDLahman']).drop('franchIDLahman', axis=1)

In [26]:
rdf = pd.read_csv('r_teams.csv', sep=' ').replace({'Milwaukee Cream Citys': 'Milwaukee Grays', 'Cleveland Forest Cities': 'Cleveland Forest Citys', 'Ft. Wayne Kekiongas': 'Fort Wayne Kekiongas', 'Baltimore Lord Baltimores': 'Baltimore Canaries', 'Worcester Brown Stockings': 'Worcester Ruby Legs', 'Richmond Virginias': 'Richmond Virginians', 'Kansas City Unions': 'Kansas City Cowboys', 'Kansas City Blues': 'Kansas City Cowboys', 'New Haven New Havens': 'New Haven Elm Citys', 'Milwaukee Unions': 'Milwaukee Brewers', 'Chicago Unions': 'Chicago Browns/Pittsburgh Stogies', 'Boston Unions': 'Boston Reds', 'Altoona Pride': 'Altoona Mountain City', 'Brooklyn Wonders': "Brooklyn Ward's Wonders", 'Philadelphia Quakers': 'Philadelphia Athletics', 'Rochester Hop Bitters': 'Rochester Broncos', 'Indianapolis Hoosier-Feds': 'Indianapolis Hoosiers', 'Buffalo Feds': 'Buffalo Blues', 'Brooklyn Feds': 'Brooklyn Tip-Tops', "Houston Colt 45's": 'Houston Colt .45s'})

rdf.loc[(rdf['season'] == 1873) & (rdf['team_full_name'] == 'Washington Nationals'), 'team_full_name'] = 'Washington Blue Legs'
rdf.loc[rdf['season'].between(1871, 1875) & (rdf['team_full_name'] == 'Boston Braves'), 'team_full_name'] = 'Boston Red Stockings'
rdf.loc[(rdf['season'] == 1875) & (rdf['team_full_name'] == 'Washington Olympics'), 'team_full_name'] = 'Washington Nationals'
rdf.loc[rdf['season'].between(1879, 1888) & (rdf['team_full_name'] == 'Cleveland Spiders'), 'team_full_name'] = 'Cleveland Blues'
rdf.loc[(rdf['season'] == 1880) & (rdf['team_full_name'] == 'Cincinnati Reds'), 'team_full_name'] = 'Cincinnati Stars'
rdf.loc[rdf['season'].between(1882, 1884) & (rdf['team_full_name'] == 'Louisville Colonels'), 'team_full_name'] = 'Louisville Eclipse'
rdf.loc[rdf['season'].between(1882, 1890) & (rdf['team_full_name'] == 'Pittsburgh Pirates'), 'team_full_name'] = 'Pittsburgh Alleghenys'
rdf.loc[(rdf['season'] == 1882) & (rdf['team_full_name'] == 'St. Louis Cardinals'), 'team_full_name'] = 'St. Louis Brown Stockings'
rdf.loc[rdf['season'].between(1883, 1898) & (rdf['team_full_name'] == 'St. Louis Cardinals'), 'team_full_name'] = 'St. Louis Browns'
rdf.loc[rdf['season'].between(1883, 1884) & (rdf['team_full_name'] == 'Columbus Colts'), 'team_full_name'] = 'Columbus Buckeyes'
rdf.loc[(rdf['season'] == 1884) & (rdf['team_full_name'] == 'Indianapolis Blues'), 'team_full_name'] = 'Indianapolis Hoosiers'
rdf.loc[(rdf['season'] == 1885) & (rdf['team_full_name'] == 'New York Gothams'), 'team_full_name'] = 'New York Giants'
rdf.loc[rdf['season'].between(1886, 1889) & (rdf['team_full_name'] == 'Washington Senators'), 'team_full_name'] = 'Washington Nationals'
rdf.loc[rdf['season'].between(1889, 1891) & (rdf['team_full_name'] == 'Columbus Colts'), 'team_full_name'] = 'Columbus Solons'
rdf.loc[(rdf['season'] == 1891) & (rdf['team_full_name'] == 'Washington Senators'), 'team_full_name'] = 'Washington Statesmen'
rdf.loc[(rdf['season'] == 1907) & (rdf['team_full_name'] == 'Boston Red Sox'), 'team_full_name'] = 'Boston Americans'
rdf.loc[(rdf['season'] == 1899) & (rdf['team_full_name'] == 'St. Louis Cardinals'), 'team_full_name'] = 'St. Louis Perfectos'
rdf.loc[(rdf['season'] == 1965) & (rdf['team_full_name'] == 'Los Angeles Angels'), 'team_full_name'] = 'California Angels'
rdf.loc[rdf['season'].between(2005, 2015) & (rdf['team_full_name'] == 'Los Angeles Angels'), 'team_full_name'] = 'Los Angeles Angels of Anaheim'
rdf['lgID'] = rdf['league_id'].map({100: 'AA', 101: 'UA', 102: 'NA', 103: 'AL', 104: 'NL', 105: 'PL', 106: 'FL'})

In [27]:
teams = teams.merge(rdf, how='left', left_on=['yearID', 'teamName', 'lgID'], right_on=['season', 'team_full_name', 'lgID'])[['yearID', 'franchID', 'lgID', 'divID', 'parkID', 'spring_league_id', 'spring_venue_id', 'teamName', 'location', 'nickname', 'teamAbrFG', 'teamAbrBR', 'teamAbrLahman', 'teamAbrLahman45', 'teamAbrRetro', 'team_abbreviation', 'team_code']].rename({'spring_league_id': 'lgIDST', 'spring_venue_id': 'parkIDST', 'team_abbreviation': 'teamAbrMLB', 'team_code': 'teamCodeMLB'}, axis=1).set_index(['yearID', 'franchID']).sort_index()
teams

lgID divID parkID  lgIDST  parkIDST                 teamName  \
yearID franchID                                                                
1871   1          NA   NaN  BOS01     NaN       NaN     Boston Red Stockings   
       2          NA   NaN  CHI01     NaN       NaN  Chicago White Stockings   
       31         NA   NaN  PHI01     NaN       NaN   Philadelphia Athletics   
       32         NA   NaN  CLE01     NaN       NaN   Cleveland Forest Citys   
       33         NA   NaN  FOR01     NaN       NaN     Fort Wayne Kekiongas   
...              ...   ...    ...     ...       ...                      ...   
2023   26         AL     E  TOR02   115.0    2536.0        Toronto Blue Jays   
       27         NL     W  DEN02   114.0    4249.0         Colorado Rockies   
       28         NL     E  MIA02   115.0    2520.0            Miami Marlins   
       29         AL     E  STP01   115.0    2534.0           Tampa Bay Rays   
       30         NL     W  PHO01   114.0    4249.0     Arizona Diamondbacks   

                     location         nickname teamAbrFG teamAbrBR  \
yearID franchID                                                      
1871   1               Boston    Red Stockings       BSN       BOS   
       2              Chicago  White Stockings       CHI       CHI   
       31        Philadelphia        Athletics       PHN       ATH   
       32           Cleveland     Forest Citys       CLE       CLE   
       33          Fort Wayne        Kekiongas       FTW       KEK   
...                       ...              ...       ...       ...   
2023   26             Toronto        Blue Jays       TOR       TOR   
       27            Colorado          Rockies       COL       COL   
       28               Miami          Marlins       MIA       MIA   
       29           Tampa Bay             Rays       TBR       TBR   
       30             Arizona     Diamondbacks       ARI       ARI   

                teamAbrLahman teamAbrLahman45 teamAbrRetro teamAbrMLB  \
yearID franchID                                                         
1871   1                  BS1             BS1          BS1        BSN   
       2                  CH1             CH1          CH1        CWS   
       31                 PH1             PH1          PH1        PHN   
       32                 CL1             CL1          CL1        CLE   
       33                 FW1             FW1          FW1        FTW   
...                       ...             ...          ...        ...   
2023   26                 TOR             TOR          TOR        TOR   
       27                 COL             COL          COL        COL   
       28                 MIA             FLO          MIA        MIA   
       29                 TBA             TBA          TBA         TB   
       30                 ARI             ARI          ARI         AZ   

                teamCodeMLB  
yearID franchID              
1871   1                bs1  
       2                ch1  
       31               ph1  
       32               cl1  
       33               fw1  
...                     ...  
2023   26               tor  
       27               col  
       28               mia  
       29               tba  
       30               ari  

[3045 rows x 15 columns]

In [28]:
franchises = franchises.replace({'franchIDFG': {1091: 1028, 1092: 9}, 'franchIDLahman': {'CNS': 'CNR', 'BLA': 'NYY'}})
franchises

,franchIDFG,franchIDBR,franchIDLahman,yearFirst,yearLast,yearsActive,franchName,isActive
franchID,,,,,,,,
1,16,ATL,ATL,1871,2023,153,Atlanta Braves,True
2,17,CHC,CHC,1871,2023,151,Chicago Cubs,True
3,18,CIN,CIN,1882,2023,142,Cincinnati Reds,True
4,27,PIT,PIT,1882,2023,142,Pittsburgh Pirates,True
5,28,STL,STL,1882,2023,142,St. Louis Cardinals,True
...,...,...,...,...,...,...,...,...
112,1021,CHH,CHH,1914,1915,2,Chicago Whales,False
113,1040,KCP,KCP,1914,1915,2,Kansas City Packers,False
114,1051,NEW,NEW,1914,1915,2,Newark Pepper,False


In [29]:
franchises.to_csv('Database/franchises.csv')
teams.to_csv('Database/teams.csv')

## Player Biographical Info

In [12]:
players_seasons_teams = pd.concat([pd.read_csv(file)[['PlayerId', 'MLBAMID', 'Name', 'Season', 'Team']].rename({'PlayerId': 'playerIDFG', 'MLBAMID': 'playerIDMLBAM', 'Name': 'playerName', 'Season': 'yearID', 'Team': 'teamAbrFG'}, axis=1).astype({'playerIDMLBAM': 'Int64'}) for file in glob.glob('Hitters/*.csv')] + [pd.read_csv(file)[['PlayerId', 'MLBAMID', 'Name', 'Season', 'Team']].rename({'PlayerId': 'playerIDFG', 'MLBAMID': 'playerIDMLBAM', 'Name': 'playerName', 'Season': 'yearID', 'Team': 'teamAbrFG'}, axis=1).astype({'playerIDMLBAM': 'Int64'}) for file in glob.glob('Pitchers/*.csv')] + [pd.read_csv(file)[['PlayerId', 'MLBAMID', 'Name', 'Season', 'Team']].rename({'PlayerId': 'playerIDFG', 'MLBAMID': 'playerIDMLBAM', 'Name': 'playerName', 'Season': 'yearID', 'Team': 'teamAbrFG'}, axis=1).astype({'playerIDMLBAM': 'Int64'}) for file in glob.glob('Fielders/*.csv')], ignore_index=True).drop_duplicates().merge(pd.read_csv('Database/teams.csv')[['yearID', 'franchID', 'teamAbrFG']], how='left', on=['yearID', 'teamAbrFG'])

dupes = players_seasons_teams[players_seasons_teams[['playerIDFG', 'yearID', 'teamAbrFG']].duplicated(keep=False)].sort_values(['yearID', 'teamAbrFG'], ignore_index=True)

browns = pd.concat([pd.read_csv(file).dropna(axis=1).drop('Pos', axis=1, errors='ignore') for file in glob.glob('St. Louis Browns/*.csv')]).drop_duplicates().rename({'PlayerId': 'playerIDFG', 'MLBAMID': 'playerIDMLBAM', 'Name': 'playerName', 'Season': 'yearID', 'Team': 'teamAbrFG'}, axis=1)
orioles = pd.concat([pd.read_csv(file).dropna(axis=1).drop('Pos', axis=1, errors='ignore') for file in glob.glob('Baltimore Orioles/*.csv')]).drop_duplicates().rename({'PlayerId': 'playerIDFG', 'MLBAMID': 'playerIDMLBAM', 'Name': 'playerName', 'Team': 'teamAbrFG'}, axis=1).assign(yearID=1884)
dupes = dupes.merge(pd.concat([browns[['yearID', 'playerIDFG']].assign(franchID=5, valid=True), orioles[['yearID', 'playerIDFG']].assign(franchID=66, valid=True)]), how='left', on=['yearID', 'playerIDFG', 'franchID']).sort_values(['yearID', 'teamAbrFG'], ignore_index=True)

players_seasons_teams = players_seasons_teams.merge(dupes.groupby(['yearID', 'playerIDFG', 'teamAbrFG'])[['franchID', 'valid']].apply(lambda x: x['franchID'].iloc[x['valid'].sum()]).reset_index().drop('teamAbrFG', axis=1).rename({0: 'franchID'}, axis=1).assign(valid=False), how='left', on=['yearID', 'playerIDFG', 'franchID']).astype({'valid': bool}).query('valid == True').drop(['teamAbrFG', 'valid'], axis=1).merge(pd.read_csv('Database/franchises.csv')[['franchID', 'franchName']], on='franchID')

players_seasons_teams.loc[players_seasons_teams['playerIDFG'] == 1012886, ['playerIDFG', 'playerName']] = [1012883, 'Live Oak Taylor']
players_seasons_teams.loc[players_seasons_teams['playerIDFG'] == 1012086, ['playerIDFG', 'playerName']] = [1012196, 'Rex Smith']
players_seasons_teams.loc[players_seasons_teams['playerIDFG'] == 1008396, ['playerIDFG', 'playerIDMLBAM', 'playerName']] = [1008397, 118581, 'Bill McCloskey']
players_seasons_teams.loc[players_seasons_teams['playerIDFG'] == 1012999, ['playerIDFG', 'playerIDMLBAM', 'playerName']] = [1013006, 123281, 'Frank Thompson']
players_seasons_teams.loc[players_seasons_teams['playerIDFG'] == 1013857, ['playerIDFG', 'playerIDMLBAM', 'playerName']] = [1013854, 124192, 'Bill White']
# players_seasons_teams.loc[players_seasons_teams['playerIDFG'] == 1013001, ['playerIDFG', 'playerIDMLBAM', 'playerName']] = [1012915, 123187, 'Fred Tenney']
players_seasons_teams.loc[players_seasons_teams['playerIDFG'] == 1009694, ['playerIDFG', 'playerIDMLBAM', 'playerName']] = [1009698, 119902, "Pete O'Brien"]
players_seasons_teams.loc[players_seasons_teams['playerIDFG'] == 1002444, ['playerIDFG', 'playerIDMLBAM', 'playerName']] = [1002445, 112521, 'Bill Collins']
players_seasons_teams.loc[players_seasons_teams['playerIDFG'] == 1012419, ['playerIDFG', 'playerIDMLBAM', 'playerName']] = [1012422, 122680, 'Tom Stanton']

MISSING_MLBAMS = {1000511: 110535, 1000769: 110789, 1000777: 110797, 1000778: 110798, 1001791: 111832, 1003245: 113308, 1004075: 114157, 1004972: 115064, 1005919: 116046, 1006675: 116818, 1007769: 117936, 1007906: 118075, 1008462: 118644, 1009200: 119402, 1009235: 119428, 1009111: 119301, 1009203: 119396, 1009694: 119902, 1009698: 119902, 1010104: 120330, 1010128: 120356, 1010610: 120846, 1010739: 120968, 1010882: 121130, 1011053: 121323, 1012886: 123141, 1013857: 124192, 1014429: 124756, 1014468: 124756}
BREF_NO_MLBAM = {1003504: 'driscde02', 1009181: 'morgapi01', 1010606: 'quinnjo01', 1002190: 'childsa01', 1014137: 'winklge01', 1007482: 'lehanja01', 1010609: 'quinnpa01', 1014470: 'gormato04', 1011344: 'ryanjo03', 1014471: 'mccaf01', 1009686: "o'brida01", 1012086: 'smithre01', 1014345: 'yinglch01', 1012879: 'tayloja02', 1014469: 'glaseno01'}

PlayerKeys = namedtuple('PlayerKeys', ['key_mlbam', 'key_fangraphs', 'key_bbref'])

sparrow_morton = PlayerKeys(119428, 1009235, 'mortosp01')
henry_moore = PlayerKeys(119301, 1009111, 'mooreha01')
bob_hogan = PlayerKeys(116046, 1005918, 'hoganed01')
mortimer_hogan = PlayerKeys(654574, 1005919, 'hoganed02')
live_oak_taylor = PlayerKeys([123135, 123141], [1012883, 1012886], 'tayloli01')  # Live Oak and Sandy Taylor are the same person
james_morris = PlayerKeys(119396, 1009203, 'morrie.01')
denny_driscoll = PlayerKeys(113570, 1003503, 'driscde01')
dennis_driscoll = PlayerKeys(np.nan, 1003504, 'driscde02')
bill_d_white = PlayerKeys(124192, 1013854, 'whitebi02')
bill_e_white = PlayerKeys(124193, 1013853, 'whitebi01')
rex_smith = PlayerKeys([np.nan, 122446], [1012086, 1012196], 'smithre01')  # (Unknown Smith) and Rex Smith are the same person
james_cavanagh = PlayerKeys(116867, 1006719, 'kavan01')
jim_donnelly = PlayerKeys(137200, 1003385, 'donneji02')
ben_harrison = PlayerKeys(115541, 1005394, 'harribe01')

pid = pd.read_csv('playerIDlookup.csv', delimiter=' ', low_memory=False).drop(['key_sr_nfl', 'key_sr_nba', 'key_sr_nhl', 'key_findagrave'], axis=1).query('mlb_played_first.notna()')

pid.loc[pid['key_fangraphs'] == 1014468, 'key_mlbam'] = 1014468
pid.loc[pid['key_fangraphs'] == 1005919, 'key_mlbam'] = 654574
pid.loc[pid['key_mlbam'] == 119428, 'key_fangraphs'] = 1009235
pid.loc[pid['key_mlbam'] == 119301, 'key_fangraphs'] = 1009111
pid.loc[pid['key_mlbam'] == 1005919, 'key_fangraphs'] = 1009203
pid.loc[pid['key_mlbam'] == 116867, 'key_fangraphs'] = 1006719
pid.loc[pid['key_mlbam'] == 137200, 'key_fangraphs'] = 1003385
pid.loc[pid['key_mlbam'] == 110535, 'key_fangraphs'] = 1000511
pid.loc[pid['key_mlbam'] == 110789, 'key_fangraphs'] = 1000769
pid.loc[pid['key_mlbam'] == 110797, 'key_fangraphs'] = 1000777
pid.loc[pid['key_mlbam'] == 110798, 'key_fangraphs'] = 1000778
pid.loc[pid['key_mlbam'] == 111832, 'key_fangraphs'] = 1001791
pid.loc[pid['key_mlbam'] == 113308, 'key_fangraphs'] = 1003245
pid.loc[pid['key_mlbam'] == 116818, 'key_fangraphs'] = 1006675
pid.loc[pid['key_mlbam'] == 117936, 'key_fangraphs'] = 1007769
pid.loc[pid['key_mlbam'] == 118075, 'key_fangraphs'] = 1007906
pid.loc[pid['key_mlbam'] == 118644, 'key_fangraphs'] = 1008462
pid.loc[pid['key_mlbam'] == 119402, 'key_fangraphs'] = 1009200
pid.loc[pid['key_mlbam'] == 119396, 'key_fangraphs'] = 1009203
pid.loc[pid['key_mlbam'] == 120330, 'key_fangraphs'] = 1010104
pid.loc[pid['key_mlbam'] == 120356, 'key_fangraphs'] = 1010128
pid.loc[pid['key_mlbam'] == 120968, 'key_fangraphs'] = 1010739
pid.loc[pid['key_mlbam'] == 121130, 'key_fangraphs'] = 1010882
pid.loc[pid['key_mlbam'] == 121323, 'key_fangraphs'] = 1011053
pid.loc[pid['key_mlbam'] == 124756, 'key_fangraphs'] = 1014429
pid.loc[pid['key_mlbam'] == 115541, ['key_bbref', 'name_first']] = ['harribe01', 'Ben']
pid.loc[pid['key_bbref'] == 'ryanjo03', ['key_mlbam', 'key_fangraphs']] = [121595, 1011344]
pid.loc[pid['key_bbref'] == 'donnejo01', 'key_fangraphs'] = 1003386

pid = pd.concat([pid, pd.DataFrame([{'key_person': np.nan, 'key_uuid': np.nan, 'key_mlbam': np.nan, 'key_retro': 'crosj102', 'key_bbref': 'crossjo01', 'key_bbref_minors': np.nan, 'key_fangraphs': np.nan, 'key_npb': np.nan, 'name_last': 'Cross', 'name_first': 'Joe', 'name_given': 'Joseph A.', 'name_suffix': np.nan, 'name_matrilineal': np.nan, 'name_nick': np.nan, 'birth_year': 1858, 'birth_month': 1, 'birth_day': 6, 'death_year': 1933, 'death_month': 4, 'death_day': 6, 'pro_played_first': 1888, 'pro_played_last': 1888, 'mlb_played_first': 1888, 'mlb_played_last': 1888, 'col_played_first': np.nan, 'col_played_last': np.nan, 'pro_managed_first': np.nan, 'pro_managed_last': np.nan, 'mlb_managed_first': np.nan, 'mlb_managed_last': np.nan, 'col_managed_first': np.nan, 'col_managed_last': np.nan, 'pro_umpired_first': np.nan, 'pro_umpired_last': np.nan, 'mlb_umpired_first': np.nan, 'mlb_umpired_last': np.nan}, {'key_person': np.nan, 'key_uuid': np.nan, 'key_mlbam': np.nan, 'key_retro': 'gilgh101', 'key_bbref': 'gilgahu01', 'key_bbref_minors': np.nan, 'key_fangraphs': np.nan, 'key_npb': np.nan, 'name_last': 'Gilgan', 'name_first': 'Hugh', 'name_given': 'Hugh J.', 'name_suffix': np.nan, 'name_matrilineal': np.nan, 'name_nick': np.nan, 'birth_year': 1852, 'birth_month': np.nan, 'birth_day': np.nan, 'death_year': 1887, 'death_month': 6, 'death_day': 17, 'pro_played_first': 1875, 'pro_played_last': 1875, 'mlb_played_first': 1875, 'mlb_played_last': 1875, 'col_played_first': np.nan, 'col_played_last': np.nan, 'pro_managed_first': np.nan, 'pro_managed_last': np.nan, 'mlb_managed_first': np.nan, 'mlb_managed_last': np.nan, 'col_managed_first': np.nan, 'col_managed_last': np.nan, 'pro_umpired_first': np.nan, 'pro_umpired_last': np.nan, 'mlb_umpired_first': np.nan, 'mlb_umpired_last': np.nan}, {'key_person': np.nan, 'key_uuid': np.nan, 'key_mlbam': np.nan, 'key_retro': 'joned108', 'key_bbref': 'jonesda06', 'key_bbref_minors': np.nan, 'key_fangraphs': np.nan, 'key_npb': np.nan, 'name_last': 'Jones', 'name_first': 'David', 'name_given': 'David E.', 'name_suffix': np.nan, 'name_matrilineal': np.nan, 'name_nick': np.nan, 'birth_year': 1861, 'birth_month': 4, 'birth_day': 5, 'death_year': 1937, 'death_month': 5, 'death_day': 1, 'pro_played_first': 1882, 'pro_played_last': 1882, 'mlb_played_first': 1882, 'mlb_played_last': 1882, 'col_played_first': np.nan, 'col_played_last': np.nan, 'pro_managed_first': np.nan, 'pro_managed_last': np.nan, 'mlb_managed_first': np.nan, 'mlb_managed_last': np.nan, 'col_managed_first': np.nan, 'col_managed_last': np.nan, 'pro_umpired_first': np.nan, 'pro_umpired_last': np.nan, 'mlb_umpired_first': np.nan, 'mlb_umpired_last': np.nan}, {'key_person': np.nan, 'key_uuid': np.nan, 'key_mlbam': np.nan, 'key_retro': 'mckee102', 'key_bbref': 'mckenfr01', 'key_bbref_minors': np.nan, 'key_fangraphs': np.nan, 'key_npb': np.nan, 'name_last': 'McKenna', 'name_first': 'Frank', 'name_given': 'Frank', 'name_suffix': np.nan, 'name_matrilineal': np.nan, 'name_nick': np.nan, 'birth_year': np.nan, 'birth_month': np.nan, 'birth_day': np.nan, 'death_year': np.nan, 'death_month': np.nan, 'death_day': np.nan, 'pro_played_first': 1874, 'pro_played_last': 1874, 'mlb_played_first': 1874, 'mlb_played_last': 1874, 'col_played_first': np.nan, 'col_played_last': np.nan, 'pro_managed_first': np.nan, 'pro_managed_last': np.nan, 'mlb_managed_first': np.nan, 'mlb_managed_last': np.nan, 'col_managed_first': np.nan, 'col_managed_last': np.nan, 'pro_umpired_first': np.nan, 'pro_umpired_last': np.nan, 'mlb_umpired_first': np.nan, 'mlb_umpired_last': np.nan}, {'key_person': np.nan, 'key_uuid': np.nan, 'key_mlbam': np.nan, 'key_retro': 'mckep101', 'key_bbref': 'mckenpa01', 'key_bbref_minors': np.nan, 'key_fangraphs': np.nan, 'key_npb': np.nan, 'name_last': 'McKenna', 'name_first': 'Patrick', 'name_given': 'Patrick J.', 'name_suffix': np.nan, 'name_matrilineal': np.nan, 'name_nick': np.nan, 'birth_year': 1854, 'birth_month': 12, 'birth_day': 27, 'death_year': 1922, 'death_month': 10, 'death_day': 27, 'pro_played_first': 1877, 'pro_played_last': 1877, 'mlb_played_first': 1877, 'mlb_played_last': 1877, 'col_played_first': np.nan, 'col_played_last': np.nan, 'pro_managed_first': np.nan, 'pro_managed_last': np.nan, 'mlb_managed_first': np.nan, 'mlb_managed_last': np.nan, 'col_managed_first': np.nan, 'col_managed_last': np.nan, 'pro_umpired_first': np.nan, 'pro_umpired_last': np.nan, 'mlb_umpired_first': np.nan, 'mlb_umpired_last': np.nan}, {'key_person': np.nan, 'key_uuid': np.nan, 'key_mlbam': np.nan, 'key_retro': 'shaff101', 'key_bbref': 'shafffr01', 'key_bbref_minors': np.nan, 'key_fangraphs': np.nan, 'key_npb': np.nan, 'name_last': 'Shaffer', 'name_first': 'Frank', 'name_given': 'Francis X.', 'name_suffix': np.nan, 'name_matrilineal': np.nan, 'name_nick': np.nan, 'birth_year': 1859, 'birth_month': 12, 'birth_day': 6, 'death_year': 1939, 'death_month': 3, 'death_day': 18, 'pro_played_first': 1884, 'pro_played_last': 1884, 'mlb_played_first': 1884, 'mlb_played_last': 1884, 'col_played_first': np.nan, 'col_played_last': np.nan, 'pro_managed_first': np.nan, 'pro_managed_last': np.nan, 'mlb_managed_first': np.nan, 'mlb_managed_last': np.nan, 'col_managed_first': np.nan, 'col_managed_last': np.nan, 'pro_umpired_first': np.nan, 'pro_umpired_last': np.nan, 'mlb_umpired_first': np.nan, 'mlb_umpired_last': np.nan}, {'key_person': np.nan, 'key_uuid': np.nan, 'key_mlbam': np.nan, 'key_retro': 'snydc103', 'key_bbref': 'snydech03', 'key_bbref_minors': np.nan, 'key_fangraphs': np.nan, 'key_npb': np.nan, 'name_last': 'Snyder', 'name_first': 'Chubby', 'name_given': 'Alfred Joseph', 'name_suffix': np.nan, 'name_matrilineal': np.nan, 'name_nick': np.nan, 'birth_year': 1890, 'birth_month': 8, 'birth_day': 20, 'death_year': 1954, 'death_month': 3, 'death_day': 24, 'pro_played_first': 1914, 'pro_played_last': 1914, 'mlb_played_first': 1914, 'mlb_played_last': 1914, 'col_played_first': np.nan, 'col_played_last': np.nan, 'pro_managed_first': np.nan, 'pro_managed_last': np.nan, 'mlb_managed_first': np.nan, 'mlb_managed_last': np.nan, 'col_managed_first': np.nan, 'col_managed_last': np.nan, 'pro_umpired_first': np.nan, 'pro_umpired_last': np.nan, 'mlb_umpired_first': np.nan, 'mlb_umpired_last': np.nan}, {'key_person': np.nan, 'key_uuid': np.nan, 'key_mlbam': np.nan, 'key_retro': 'sullw101', 'key_bbref': 'sulliwi01', 'key_bbref_minors': np.nan, 'key_fangraphs': np.nan, 'key_npb': np.nan, 'name_last': 'Sullivan', 'name_first': 'William', 'name_given': 'William F.', 'name_suffix': np.nan, 'name_matrilineal': np.nan, 'name_nick': np.nan, 'birth_year': 1864, 'birth_month': 3, 'birth_day': 12, 'death_year': 1911, 'death_month': 9, 'death_day': 27, 'pro_played_first': 1884, 'pro_played_last': 1884, 'mlb_played_first': 1884, 'mlb_played_last': 1884, 'col_played_first': np.nan, 'col_played_last': np.nan, 'pro_managed_first': np.nan, 'pro_managed_last': np.nan, 'mlb_managed_first': np.nan, 'mlb_managed_last': np.nan, 'col_managed_first': np.nan, 'col_managed_last': np.nan, 'pro_umpired_first': np.nan, 'pro_umpired_last': np.nan, 'mlb_umpired_first': np.nan, 'mlb_umpired_last': np.nan}, {'key_person': np.nan, 'key_uuid': np.nan, 'key_mlbam': np.nan, 'key_retro': 'tinnt101', 'key_bbref': 'tinneth01', 'key_bbref_minors': np.nan, 'key_fangraphs': np.nan, 'key_npb': np.nan, 'name_last': 'Tinney', 'name_first': 'Thomas', 'name_given': 'Thomas Brown', 'name_suffix': np.nan, 'name_matrilineal': np.nan, 'name_nick': np.nan, 'birth_year': 1855, 'birth_month': np.nan, 'birth_day': np.nan, 'death_year': 1905, 'death_month': 5, 'death_day': 10, 'pro_played_first': 1884, 'pro_played_last': 1884, 'mlb_played_first': 1884, 'mlb_played_last': 1884, 'col_played_first': np.nan, 'col_played_last': np.nan, 'pro_managed_first': np.nan, 'pro_managed_last': np.nan, 'mlb_managed_first': np.nan, 'mlb_managed_last': np.nan, 'col_managed_first': np.nan, 'col_managed_last': np.nan, 'pro_umpired_first': np.nan, 'pro_umpired_last': np.nan, 'mlb_umpired_first': np.nan, 'mlb_umpired_last': np.nan}, {'key_person': np.nan, 'key_uuid': np.nan, 'key_mlbam': np.nan, 'key_retro': 'zeihh101', 'key_bbref': 'zeihehe01', 'key_bbref_minors': np.nan, 'key_fangraphs': np.nan, 'key_npb': np.nan, 'name_last': 'Zeiher', 'name_first': 'Henry', 'name_given': 'Henry', 'name_suffix': np.nan, 'name_matrilineal': np.nan, 'name_nick': np.nan, 'birth_year': 1862, 'birth_month': 8, 'birth_day': 11, 'death_year': 1951, 'death_month': 10, 'death_day': 14, 'pro_played_first': 1886, 'pro_played_last': 1886, 'mlb_played_first': 1886, 'mlb_played_last': 1886, 'col_played_first': np.nan, 'col_played_last': np.nan, 'pro_managed_first': np.nan, 'pro_managed_last': np.nan, 'mlb_managed_first': np.nan, 'mlb_managed_last': np.nan, 'col_managed_first': np.nan, 'col_managed_last': np.nan, 'pro_umpired_first': np.nan, 'pro_umpired_last': np.nan, 'mlb_umpired_first': np.nan, 'mlb_umpired_last': np.nan}])])

players_seasons_teams = players_seasons_teams.merge(pid[['key_fangraphs', 'key_bbref']].rename({'key_fangraphs': 'playerIDFG', 'key_bbref': 'playerIDBR'}, axis=1), how='left', on='playerIDFG')
players_seasons_teams = players_seasons_teams[(players_seasons_teams['playerIDFG'] != 1003385) | ((players_seasons_teams['playerIDBR'] == 'donneji01') & (players_seasons_teams['franchID'] != 77)) | ((players_seasons_teams['playerIDBR'] == 'donneji02') & (players_seasons_teams['franchID'] == 77))]
players_seasons_teams = pd.concat([players_seasons_teams.loc[~players_seasons_teams[['playerIDFG', 'yearID', 'franchID']].apply(tuple, axis=1).isin([(1004721, 1875, 39), (1006527, 1882, 66), (1008591, 1874, 44), (1008591, 1877, 50), (1012248, 1914, 109), (1013900, 1886, 88), (1013001, 1884, 85), (1005252, 1875, 52)]), ['playerIDFG', 'playerIDBR', 'playerIDMLBAM', 'playerName', 'yearID', 'franchID', 'franchName']], pd.DataFrame([{'playerIDFG': np.nan, 'playerIDBR': 'crossjo01', 'playerIDMLBAM': np.nan, 'playerName': 'Joe Cross', 'yearID': 1888, 'franchID': 67, 'franchName': 'Louisville Colonels'}, {'playerIDFG': np.nan, 'playerIDBR': 'gilgahu01', 'playerIDMLBAM': np.nan, 'playerName': 'Hugh Gilgan', 'yearID': 1875, 'franchID': 39, 'franchName': 'Brooklyn Atlantics'}, {'playerIDFG': np.nan, 'playerIDBR': 'jonesda06', 'playerIDMLBAM': np.nan, 'playerName': 'David Jones', 'yearID': 1882, 'franchID': 66, 'franchName': 'Baltimore Orioles'}, {'playerIDFG': np.nan, 'playerIDBR': 'mckenfr01', 'playerIDMLBAM': np.nan, 'playerName': 'Frank McKenna', 'yearID': 1874, 'franchID': 44, 'franchName': 'Philadelphia White Stockings'}, {'playerIDFG': np.nan, 'playerIDBR': 'mckenpa01', 'playerIDMLBAM': np.nan, 'playerName': 'Patrick McKenna', 'yearID': 1877, 'franchID': 50, 'franchName': 'St. Louis Brown Stockings'}, {'playerIDFG': np.nan, 'playerIDBR': 'shafffr01', 'playerIDMLBAM': np.nan, 'playerName': 'Frank Shaffer', 'yearID': 1884, 'franchID': 71, 'franchName': 'Altoona Mountain Citys'}, {'playerIDFG': np.nan, 'playerIDBR': 'snydech03', 'playerIDMLBAM': np.nan, 'playerName': 'Chubby Snyder', 'yearID': 1914, 'franchID': 109, 'franchName': 'Buffalo Bisons'}, {'playerIDFG': np.nan, 'playerIDBR': 'sulliwi01', 'playerIDMLBAM': np.nan, 'playerName': 'William Sullivan', 'yearID': 1884, 'franchID': 81, 'franchName': 'St. Louis Maroons'}, {'playerIDFG': np.nan, 'playerIDBR': 'tinneth01', 'playerIDMLBAM': np.nan, 'playerName': 'Thomas Tinney', 'yearID': 1884, 'franchID': 86, 'franchName': 'Washington Nationals'}, {'playerIDFG': np.nan, 'playerIDBR': 'zeihehe01', 'playerIDMLBAM': np.nan, 'playerName': 'Henry Zeiher', 'yearID': 1886, 'franchID': 88, 'franchName': 'Washington Nationals'}])], ignore_index=True)

# players_seasons_teams = players_seasons_teams.sort_values(['yearID', 'franchID', 'playerIDFG'], ignore_index=True)
# players_seasons_teams = players_seasons_teams.merge(players_seasons_teams['playerIDFG'].drop_duplicates().to_frame().set_index(pd.RangeIndex(start=1, stop=players_seasons_teams['playerIDFG'].nunique() + 1, name='playerID')).reset_index(), on='playerIDFG')[['yearID', 'franchID', 'playerID', 'playerIDFG', 'playerIDMLBAM', 'playerName', 'franchName']]
missing = pid.rename({'key_fangraphs': 'playerIDFG'}, axis=1).loc[~pid['key_fangraphs'].isin(players_seasons_teams['playerIDFG']), 'playerIDFG'].astype(int).tolist()
players_seasons_teams
# TODO: Deal with Joe/Lave Cross in FanGraphs data on the 1888 Louisville Colonels (67)*
# TODO: Deal with Hugh Gilgan in FanGraphs data on the 1875 Brooklyn Atlantics (39); stats currently attributed to Barney Gilligan (1004721)
# TODO: Deal with David Jones in FanGraphs data on the 1882 Baltimore Orioles (66); stats currently attributed to Bill Jones (1006527)
# TODO: Deal with Frank McKenna in FanGraphs data on the 1874 Philadelphia White Stockings (44); stats currently attributed to Ed McKenna (1008591)
# TODO: Deal with Patrick McKenna in FanGraphs data on the 1877 St. Louis Brown Stockings (50); stats currently attributed to Ed McKenna (1008591)
# TODO: Deal with Frank Shaffer in FanGraphs data on the 1884 Altoona Mountain Citys (); stats currently attributed to Taylor Shafer (1011784)*
# TODO: Deal with Chubby Snyder in FanGraphs data on the 1914 Buffalo Bisons; stats currently attributed to Jack Snyder (1012248)
# TODO: Deal with William Sullivan in FanGraphs data on the 1884 St. Louis Maroons; stats currently attributed to Sleeper Sullivan (1012711)*
# TODO: Deal with Thomas Tinney in FanGraphs data on the 1884 Washington Nationals; stats currently attributed to Fred Tenney (1012915)*
# TODO: Deal with Henry Zeiher in FanGraphs data on the 1886 Washington Nationals; stats currently attributed to Ed Whiting (1013900)

/var/folders/cf/xvbkm9r16gb4zfcs0bxqrd7m0000gn/T/ipykernel_2486/3996833008.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  players_seasons_teams = pd.concat([players_seasons_teams.loc[~players_seasons_teams[['playerIDFG', 'yearID', 'franchID']].apply(tuple, axis=1).isin([(1004721, 1875, 39), (1006527, 1882, 66), (1008591, 1874, 44), (1008591, 1877, 50), (1012248, 1914, 109), (1013900, 1886, 88), (1013001, 1884, 85), (1005252, 1875, 52)]), ['playerIDFG', 'playerIDBR', 'playerIDMLBAM', 'playerName', 'yearID', 'franchID', 'franchName']], pd.DataFrame([{'playerIDFG': np.nan, 'playerIDBR': 'crossjo01', 'playerIDMLBAM': np.nan, 'playerName': 'Joe Cross', 'yearID': 1888, 'franchID': 67, 'franchName': 'Louisville Colonels'}, {'playerIDFG': np.

,playerIDFG,playerIDBR,playerIDMLBAM,playerName,yearID,franchID,franchName
0,13611.0,bettsmo01,605141,Mookie Betts,2016,10,Boston Red Sox
1,5417.0,altuvjo01,514888,Jose Altuve,2015,19,Houston Astros
2,5417.0,altuvjo01,514888,Jose Altuve,2016,19,Houston Astros
3,4922.0,inciaen01,542255,Ender Inciarte,2017,1,Atlanta Braves
4,11281.0,merriwh01,593160,Whit Merrifield,2021,21,Kansas City Royals
...,...,...,...,...,...,...,...
113615,NaN,shafffr01,<NA>,Frank Shaffer,1884,71,Altoona Mountain Citys
113616,NaN,snydech03,<NA>,Chubby Snyder,1914,109,Buffalo Bisons
113617,NaN,sulliwi01,<NA>,William Sullivan,1884,81,St. Louis Maroons
113618,NaN,tinneth01,<NA>,Thomas Tinney,1884,86,Washington Nationals


In [89]:
players_seasons_teams[(players_seasons_teams['yearID'] == 1882) & (players_seasons_teams['franchID'] == 67)]
# players_seasons_teams.loc[players_seasons_teams['yearID'] == 1882, 'franchID'].value_counts()

,playerIDFG,playerIDBR,playerIDMLBAM,playerName,yearID,franchID,franchName
81861,1001128.0,bohnch01,111157,Charlie Bohn,1882,67,Louisville Colonels
81864,1001191.0,bootham01,111223,Amos Booth,1882,67,Louisville Colonels
81869,1001554.0,brownpe01,111610,Pete Browning,1882,67,Louisville Colonels
81893,1002812.0,crottjo01,112871,Joe Crotty,1882,67,Louisville Colonels
81909,1003620.0,dylerjo01,113687,John Dyler,1882,67,Louisville Colonels
81942,1005573.0,heckegu01,115688,Guy Hecker,1882,67,Louisville Colonels
81972,1007915.0,mackde01,118083,Denny Mack,1882,67,Louisville Colonels
81979,1008213.0,maskrha01,118393,Harry Maskrey,1882,67,Louisville Colonels
81980,1008214.0,maskrle01,118394,Leech Maskrey,1882,67,Louisville Colonels
81983,1008353.0,mccafha01,118534,Harry McCaffery,1882,67,Louisville Colonels


In [17]:
bref = pd.concat([pd.read_csv('bref_standard_batting.csv')[['PlayerName', 'PlayerID', 'Season', 'Team', 'PA']], pd.read_csv('bref_standard_pitching.csv')[['PlayerName', 'PlayerID', 'Season', 'Team']], pd.read_csv('bref_standard_fielding.csv')[['PlayerName', 'PlayerID', 'Season', 'Team']]], ignore_index=True).rename({'PlayerName': 'playerName', 'PlayerID': 'playerIDBR', 'Season': 'yearID', 'Team': 'teamAbrBR'}, axis=1).drop_duplicates(ignore_index=True).merge(pid[['key_fangraphs', 'key_bbref', 'key_mlbam']].rename({'key_fangraphs': 'playerIDFG', 'key_bbref': 'playerIDBR', 'key_mlbam': 'playerIDMLBAM'}, axis=1), on='playerIDBR').merge(pd.read_csv('Database/teams.csv')[['yearID', 'franchID', 'teamAbrBR']], on=['yearID', 'teamAbrBR'])[['playerIDFG', 'playerIDBR', 'playerIDMLBAM', 'playerName', 'yearID', 'franchID', 'teamAbrBR', 'PA']].drop_duplicates(subset=['playerIDFG', 'playerIDBR', 'playerIDMLBAM', 'playerName', 'yearID', 'franchID', 'teamAbrBR'])
bref

,playerIDFG,playerIDBR,playerIDMLBAM,playerName,yearID,franchID,teamAbrBR,PA
0,1000017.0,abercda01,110018.0,Frank Abercrombie,1871,37,TRO,4.0
1,1000070.0,addybo01,110074.0,Bob Addy,1871,36,ROK,122.0
2,1000164.0,allisar01,110170.0,Art Allison,1871,32,CLE,139.0
3,1000168.0,allisdo01,110172.0,Doug Allison,1871,35,OLY,133.0
4,1000272.0,ansonca01,110284.0,Cap Anson,1871,36,ROK,122.0
...,...,...,...,...,...,...,...,...
154881,15094.0,zastrro01,642239.0,Rob Zastryzny,2023,4,PIT,NaN
154883,22717.0,zerpaan01,672582.0,Angel Zerpa,2023,21,KCR,NaN
154884,20370.0,zimmebr02,669145.0,Bruce Zimmermann,2023,9,BAL,NaN
154885,19588.0,zuniggu01,670871.0,Guillermo Zuñiga,2023,5,STL,NaN


In [29]:
len(set(bref['playerIDBR'].dropna().unique())), len(set(pid['key_bbref'].dropna().unique()))

(20729, 23114)

In [21]:
bref.merge(players_seasons_teams, how='left', on=['playerIDFG', 'playerIDBR', 'playerIDMLBAM', 'yearID', 'franchID'], suffixes=('BR', 'FG')).query('playerNameFG.isna() & PA == 0')

,playerIDFG,playerIDBR,playerIDMLBAM,playerNameBR,yearID,franchID,teamAbrBR,PA,playerNameFG,franchName
1570,1008786.0,merried01,118972.0,Ed Merrill,1882,67,LOU,0.0,NaN,NaN
3263,1006632.0,joyce01,116775.0,George Joyce,1886,88,WHS,0.0,NaN,NaN
3543,1004186.0,flynnjo02,114267.0,Jocko Flynn,1887,2,CHC,0.0,NaN,NaN
8760,1007984.0,maharfr01,118156.0,Frank Mahar,1902,6,PHI,0.0,NaN,NaN
9882,1003916.0,falloch01,113992.0,Charlie Fallon,1905,16,NYY,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
45983,1005373.0,harmote01,115486.0,Terry Harmon,1967,6,PHI,0.0,NaN,NaN
46144,1008136.0,marshda01,118315.0,Dave Marshall,1967,7,SFG,0.0,NaN,NaN
48688,1008486.0,mcfadle01,118669.0,Leon McFadden,1970,19,HOU,0.0,NaN,NaN
49852,1012852.0,taverfr01,123121.0,Frank Taveras,1971,4,PIT,0.0,NaN,NaN


In [106]:
pd.read_csv('bref_standard_batting.csv').query('PA + R == 0').merge(pd.read_csv('bref_standard_pitching.csv')[['PlayerName', 'PlayerID', 'Season', 'Team', 'League', 'BF']], how='left', on=['PlayerName', 'PlayerID', 'Season', 'Team', 'League']).query('BF.isna()').drop('BF', axis=1).merge(pd.read_csv('bref_standard_fielding.csv')[['PlayerName', 'PlayerID', 'Season', 'Team', 'League', 'Inn']], how='left', on=['PlayerName', 'PlayerID', 'Season', 'Team', 'League']).query('Inn.isna()')

,PlayerName,PlayerID,Season,Team,League,Throws,G,PA,AB,R,...,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Position,Inn
0,Ed Merrill,merried01,1882,LOU,AA,NaN,1,0,0,0,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,/O,NaN
5,Charlie Bastian,bastich01,1891,PHI,NL,R,1,0,0,0,...,NaN,NaN,0,NaN,0.0,NaN,NaN,NaN,/6,NaN
10,Jiggs Donahue,donahji02,1901,PIT,NL,L,2,0,0,0,...,NaN,NaN,0,NaN,0.0,0.0,NaN,NaN,/27,NaN
12,Doc Marshall,marshdo01,1904,NYG,NL,R,1,0,0,0,...,NaN,NaN,0,NaN,0.0,0.0,NaN,NaN,/H2479,NaN
13,Frank McManus,mcmanfr01,1904,DET,AL,R,1,0,0,0,...,NaN,NaN,0,NaN,0.0,0.0,NaN,NaN,/2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,Ian Miller,milleia01,2020,CHC,NL,L,1,0,0,0,...,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,/H,NaN
275,Rylan Bannon,bannory01,2022,ATL,NL,R,1,0,0,0,...,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,/4,NaN
277,Matt Reynolds,reynoma03,2022,NYM,NL,R,1,0,0,0,...,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,/9,NaN
278,Yolmer Sanchez,sanchca01,2022,NYM,NL,B,3,0,0,0,...,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,/5H6,NaN


In [90]:
bref.merge(players_seasons_teams, how='left', on=['playerIDFG', 'playerIDBR', 'playerIDMLBAM', 'yearID', 'franchID'], suffixes=('BR', 'FG')).query('yearID == 1882 & franchID == 67')
# bref.merge(players_seasons_teams, how='left', on=['playerIDFG', 'playerIDBR', 'playerIDMLBAM', 'yearID', 'franchID'], suffixes=('BR', 'FG')).query('yearID == 1882')['franchID'].value_counts()

,playerIDFG,playerIDBR,playerIDMLBAM,playerNameBR,yearID,franchID,teamAbrBR,playerNameFG,franchName
1437,1001128.0,bohnch01,111157.0,Charlie Bohn,1882,67,LOU,Charlie Bohn,Louisville Colonels
1440,1001191.0,bootham01,111223.0,Amos Booth,1882,67,LOU,Amos Booth,Louisville Colonels
1446,1001554.0,brownpe01,111610.0,Pete Browning,1882,67,LOU,Pete Browning,Louisville Colonels
1468,1002812.0,crottjo01,112871.0,Joe Crotty,1882,67,LOU,Joe Crotty,Louisville Colonels
1485,1003620.0,dylerjo01,113687.0,John Dyler,1882,67,LOU,John Dyler,Louisville Colonels
1518,1005573.0,heckegu01,115688.0,Guy Hecker,1882,67,LOU,Guy Hecker,Louisville Colonels
1549,1007915.0,mackde01,118083.0,Denny Mack,1882,67,LOU,Denny Mack,Louisville Colonels
1556,1008213.0,maskrha01,118393.0,Harry Maskrey,1882,67,LOU,Harry Maskrey,Louisville Colonels
1557,1008214.0,maskrle01,118394.0,Leech Maskrey,1882,67,LOU,Leech Maskrey,Louisville Colonels
1560,1008353.0,mccafha01,118534.0,Harry McCaffery,1882,67,LOU,Harry McCaffery,Louisville Colonels


In [95]:
players_seasons_teams['yearID'].value_counts().sort_index() == bref.loc[bref['PA'].isna() | (bref['PA'] > 0), 'yearID'].value_counts().sort_index()

yearID
1871     True
1872     True
1873     True
1874     True
1875     True
        ...  
2019    False
2020    False
2021    False
2022    False
2023    False
Name: count, Length: 153, dtype: bool

In [29]:
tdf = pd.read_html(io=StringIO(str(str(BeautifulSoup([comment.strip() for comment in BeautifulSoup(requests.get('https://www.baseball-reference.com/leagues/majors/1871-standard-batting.shtml').content, 'html.parser', from_encoding='utf_8').find_all(string=lambda text: isinstance(text, Comment)) if 'table' in comment][0]).find('table')).replace('\xa0', ' ').replace('*', '').replace('#', ''))), index_col='Rk', extract_links='body')[0].drop(('Rk', None)).reset_index(drop=True)
tdf = pd.concat([pd.DataFrame(tdf['Name'].tolist(), columns=['PlayerName', 'PlayerID']), tdf.drop(['Name', 'Age'], axis=1).apply(lambda col: col.str[0])], axis=1).query("Tm != 'TOT'")
tdf

,PlayerName,PlayerID,Tm,Lg,G,PA,AB,R,H,2B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary
0,Frank Abercrombie?,/players/a/abercda01.shtml,TRO,NA,1,4,4,0,0,0,...,.000,.000,-100,0,0,,,,,/6
1,Bob Addy,/players/a/addybo01.shtml,ROK,NA,25,122,118,30,32,6,...,.322,.617,78,38,0,,,,,4/6
2,Art Allison?,/players/a/allisar01.shtml,CLE,NA,29,139,137,28,40,4,...,.394,.696,100,54,1,,,,,O/4
3,Doug Allison,/players/a/allisdo01.shtml,OLY,NA,27,133,133,28,44,10,...,.481,.812,133,64,0,,,,,2
4,Cap Anson,/players/a/ansonca01.shtml,ROK,NA,25,122,120,29,39,11,...,.467,.803,128,56,0,,,,,5/243O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,George Wright,/players/w/wrighge01.shtml,BOS,NA,16,86,80,33,33,7,...,.625,1.078,198,50,1,,,,,6/3
112,Harry Wright,/players/w/wrighha01.shtml,BOS,NA,31,160,147,42,44,5,...,.361,.717,102,53,0,,,,,O/16
113,Tom York,/players/y/yorkto01.shtml,TRO,NA,29,154,145,36,37,5,...,.428,.726,103,62,0,,,,,O
114,George Zettlein,/players/z/zettlge01.shtml,CHI,NA,28,130,128,23,32,3,...,.273,.535,50,35,0,,,,,1/O


In [24]:
sb.to_csv('bref_standard_batting.csv', index=False, chunksize=50000)

In [34]:
query = """SELECT *
            FROM (SELECT *
                      FROM players_seasons_teams
                      WHERE franchID IN (SELECT DISTINCT franchID
                                         FROM players_seasons_teams
                                         WHERE franchName IN ('Texas Rangers', 'Houston Astros'))) AS t;"""

ps.sqldf(query, env=locals())

,yearID,franchID,playerID,playerName,franchName
0,1961,18,9377,Chet Boak,Texas Rangers
1,1961,18,9490,Carl Bouldin,Texas Rangers
2,1961,18,9107,Harry Bright,Texas Rangers
3,1961,18,9491,Ed Brinkman,Texas Rangers
4,1961,18,8816,Pete Burnside,Texas Rangers
...,...,...,...,...,...
5317,2023,19,19783,Luis Garcia,Houston Astros
5318,2023,19,20599,Shawn Dubin,Houston Astros
5319,2023,19,20334,David Hensley,Houston Astros
5320,2023,19,20600,Grae Kessinger,Houston Astros


In [2]:
pid = pid.dropna(how='all', subset=['key_mlbam', 'key_bbref', 'key_fangraphs'])  # [['key_mlbam', 'key_bbref', 'key_fangraphs', 'name_last', 'name_first', 'name_given', 'name_suffix']]
pid.loc[pid['key_mlbam'] == 114158, 'key_mlbam'] = 114157
pid.loc[pid['key_mlbam'] == 115065, 'key_mlbam'] = 115064
pid.loc[pid['key_mlbam'] == 120844, 'key_mlbam'] = 120846

pid['FGID'] = pid['key_mlbam'].map(dict(map(reversed, MISSING_MLBAMS.items())))  #.fillna(pid['key_fangraphs']).dropna()
pid.dropna(subset=['key_fangraphs', 'FGID']).query('key_fangraphs != FGID')

,key_person,key_uuid,key_mlbam,key_retro,key_bbref,key_bbref_minors,key_fangraphs,key_npb,name_last,name_first,...,pro_managed_last,mlb_managed_first,mlb_managed_last,col_managed_first,col_managed_last,pro_umpired_first,pro_umpired_last,mlb_umpired_first,mlb_umpired_last,FGID
120701,413e6e20,413e6e20-8000-405e-8d8a-4d51471745db,119428.0,morts101,mortosp01,morton001spa,1009111.0,NaN,Morton,Sparrow,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1009235.0
136225,499e0736,499e0736-3128-4f52-a14f-f58b02ce6f97,116046.0,hogae101,hoganed01,hogan-001edd,1005918.0,NaN,Hogan,Bob,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1005919.0
328741,b14899b3,b14899b3-1457-4e49-8279-282602e82f46,123135.0,tayll102,tayloli01,taylor001liv,1012883.0,NaN,Taylor,Live Oak,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1012886.0
342934,b8d797f2,b8d797f2-914a-4498-b575-e4c0c6408e68,119301.0,moorh101,mooreha01,moore-004hen,1009203.0,NaN,Moore,Henry,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1009111.0
370834,c7e1352a,c7e1352a-fe78-468e-b413-1fdac87f0083,113570.0,drisd101,driscde01,drisco001den,1003503.0,NaN,Driscoll,Denny,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1003504.0
403690,d9863121,d9863121-7433-494a-a7ba-4eb0694c4bb9,124192.0,whitb104,whitebi02,white-003bil,1013854.0,NaN,White,Bill,...,1905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1013857.0
420569,e2982a16,e2982a16-720a-4c9c-bb14-9e915dc9cb26,122446.0,smitr106,smithre01,smith-002rex,1012196.0,NaN,Smith,Rex,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1012086.0


## Player Season Stats

In [3]:
df = bref_player_scrape()

  0%|          | 0/110 [00:00<?, ?it/s]

In [6]:
df = df.drop('PA', axis=1)

In [96]:
pid = pd.read_csv('playerIDlookup.csv', delimiter=' ').drop(['key_sr_nfl', 'key_sr_nba', 'key_sr_nhl', 'key_findagrave'], axis=1)
pid = pid.dropna(how='all', subset=['key_mlbam', 'key_bbref', 'key_fangraphs'])  # [['key_mlbam', 'key_bbref', 'key_fangraphs', 'name_last', 'name_first', 'name_given', 'name_suffix']]

MISSING_MLBAMS = {1000511: 110535, 1000769: 110789, 1000777: 110797, 1000778: 110798, 1001791: 111832, 1003245: 113308, 1003504: 113570, 1004075: 114157, 1004972: 115064, 1005919: 116046, 1006675: 116818, 1007769: 117936, 1007906: 118075, 1008462: 118644, 1009200: 119402, 1009235: 119428, 1009111: 119301, 1009203: 119396, 1009694: 119902, 1009698: 119902, 1010104: 120330, 1010128: 120356, 1010610: 120846, 1010739: 120968, 1010882: 121130, 1011053: 121323, 1012086: 122446, 1012886: 123135, 1013857: 124192, 1014429: 124756, 1014468: 124756}

pid['FGID'] = pid['key_mlbam'].map(dict(map(reversed, MISSING_MLBAMS.items()))).fillna(pid['key_fangraphs']).dropna()

/var/folders/cf/xvbkm9r16gb4zfcs0bxqrd7m0000gn/T/ipykernel_1545/1349912447.py:1: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  pid = pd.read_csv('playerIDlookup.csv', delimiter=' ').drop(['key_sr_nfl', 'key_sr_nba', 'key_sr_nhl', 'key_findagrave'], axis=1)


In [97]:
pid.dropna(subset='FGID').query('key_fangraphs != FGID')

,key_person,key_uuid,key_mlbam,key_retro,key_bbref,key_bbref_minors,key_fangraphs,key_npb,name_last,name_first,...,pro_managed_last,mlb_managed_first,mlb_managed_last,col_managed_first,col_managed_last,pro_umpired_first,pro_umpired_last,mlb_umpired_first,mlb_umpired_last,FGID
10543,05a483f8,05a483f8-3fe2-48c5-a6f5-4e488f80f3b4,117936.0,lougu101,lough01,loughr001bil,NaN,NaN,O'Loughlin,Patrick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1007769.0
108020,3a4e19ab,3a4e19ab-7778-4792-8cc8-16a4cd829659,118644.0,mcdoj106,mcdonji02,mcdona001jam,NaN,NaN,MacDonald,Malcolm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1008462.0
119027,404fcd05,404fcd05-1fb7-4238-ac18-117c41982f6a,110797.0,begle101,begleed01,bagley001ed-,NaN,NaN,Bagley,Ed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000777.0
120701,413e6e20,413e6e20-8000-405e-8d8a-4d51471745db,119428.0,morts101,mortosp01,morton001spa,1009111.0,NaN,Morton,Sparrow,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1009235.0
136225,499e0736,499e0736-3128-4f52-a14f-f58b02ce6f97,116046.0,hogae101,hoganed01,hogan-001edd,1005918.0,NaN,Hogan,Bob,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1005919.0
158820,55d9ef55,55d9ef55-0684-4132-ac39-c19b4c80c25c,120968.0,reagr101,reagari01,reagan001art,NaN,NaN,Ragan,Arthur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1010739.0
172840,5d665449,5d665449-a4be-4007-913f-f74f32080c0c,110789.0,beece102,beeched02,bieche001ed-,NaN,NaN,Biecher,Ed,...,1903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000769.0
180045,613d6a60,613d6a60-b588-4106-b3ef-0873a7894aa4,119396.0,morre103,morrie.01,morris021jam,NaN,NaN,Morris,James,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1009203.0
188752,65ec9731,65ec9731-07b4-4a79-95d5-3148e59a2bdf,120330.0,pearg101,pearcge01,pearce001geo,NaN,NaN,Pierce,George,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1010104.0
213675,73605cdb,73605cdb-0f18-495e-8739-afa5020e9651,113308.0,devij101,devinji01,diven-001fra,NaN,NaN,Diven,Frank,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1003245.0


In [66]:
BREF_NO_MLBAM = {1009181: 'morgapi01', 1010606: 'quinnjo01', 1002190: 'childsa01', 1014137: 'winklge01', 1007482: 'lehanja01', 1014470: 'gormato04', 1011344: 'ryanjo03', 1014471: 'mccaf01', 1009686: "o'brida01", 1014345: 'yinglch01', 1012879: 'tayloja02', 1014469: 'glaseno01'}

# pid = pd.read_csv('playerIDlookup.csv')
pid2 = pd.read_csv('playerids.csv').rename({'fangraphsid': 'FGID', 'mlbamid': 'MLBID'}, axis=1)
pid2['BRID'] = pid2['FGID'].map(BREF_NO_MLBAM)
pid2[pid2[['MLBID', 'BRID']].isna().all(axis=1)]

,FGID,MLBID,BRID
6763,1000511,NaN,NaN
7015,1000769,NaN,NaN
7023,1000777,NaN,NaN
7024,1000778,NaN,NaN
8010,1001791,NaN,NaN
9450,1003245,NaN,NaN
9703,1003504,NaN,NaN
12075,1005919,NaN,NaN
12819,1006675,NaN,NaN
13892,1007769,NaN,NaN


In [84]:
df[['PlayerID', 'TeamAbr']] = df[['PlayerID', 'TeamAbr']].apply(lambda x: x.str.strip('.shtml').str.strip('/2023').str.split('/').str[-1])

,PlayerID,TeamAbr
0,abramcj01,WSN
1,abreujo,HOU
2,abreuwi,BOS
3,acunaro01,ATL
4,adamewi01,MIL
...,...,...
893,zavalse01,None
894,zavalse01,CHW
895,zavalse01,ARI
896,zuninmi01,CLE


In [23]:
pss = pd.concat([pd.read_csv(file) for file in glob.glob('Hitters/*.csv')], ignore_index=True).rename({'PlayerID': 'FGID'}, axis=1)
# pss['IP'] = pss['IP'].round() + (pss['IP'] - pss['IP'].round()).div(0.3)
pss['BBE'] = pss[['GB', 'FB', 'LD']].sum(axis=1)
pss[['Pull', 'Cent', 'Oppo', 'Soft', 'Med', 'Hard']] = pss[['Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%']].mul(pss['BBE'], axis=0)
pss

,Season,Name,Team,G,AB,PA,H,1B,2B,3B,...,NameASCII,PlayerId,MLBAMID,BBE,Pull,Cent,Oppo,Soft,Med,Hard
0,2016,Mookie Betts,BOS,158,672,730,214,136,42,5,...,Mookie Betts,13611,605141.0,597.0,237.1881,209.3082,150.5037,103.6392,294.0225,199.3383
1,2015,Jose Altuve,HOU,154,638,689,200,141,40,4,...,Jose Altuve,5417,514888.0,574.0,260.2516,203.8848,109.8636,113.8242,311.7394,148.4364
2,2016,Jose Altuve,HOU,161,640,717,216,145,42,5,...,Jose Altuve,5417,514888.0,572.0,259.3448,204.1468,108.5084,77.9064,300.8148,193.2788
3,2017,Ender Inciarte,ATL,158,662,718,201,158,27,5,...,Ender Inciarte,4922,542255.0,553.0,167.6143,212.9050,172.4254,115.6323,315.0441,122.3789
4,2021,Whit Merrifield,KCR,162,664,720,184,129,42,3,...,Whit Merrifield,11281,593160.0,568.0,202.2080,183.4072,182.3848,68.3872,350.9104,148.7024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104484,1950,Wes Westrum,NYG,140,437,532,103,64,13,3,...,Wes Westrum,1013812,124121.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
104485,1950,Dick Whitman,PHI,75,132,145,33,26,7,0,...,Dick Whitman,1013904,124214.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
104486,1950,Ted Wilks,STL,18,4,4,0,0,0,0,...,Ted Wilks,1013962,124276.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
104487,1950,Johnny Wyrostek,CIN,131,509,569,145,98,34,5,...,Johnny Wyrostek,1014314,124638.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
pss.loc[pss['BBE'].notna() & (pss['Events'] - (pss['BBE'] + pss['BU']) == 6), ['Season', 'Name', 'Team', 'Events', 'BBE', 'GB', 'FB', 'LD', 'IFFB', 'BU', 'BUH']]

,Season,Name,Team,Events,BBE,GB,FB,LD,IFFB,BU,BUH
1561,2019,Tommy La Stella,LAA,270.0,262.0,119.0,87.0,56.0,9.0,2.0,1.0
1979,2021,Jorge Soler,KCR,220.0,214.0,85.0,98.0,31.0,12.0,0.0,0.0


In [44]:
df = pd.read_csv('hitters2002-2023alt.csv')
d = 6
df['BIP'] = df['AB'] + df['SF'] - df['SO'] + df['SH']
df['BBE'] = df[['GB', 'LD', 'FB']].sum(axis=1)
df.loc[df['BBE'] > df['BIP'], ['Season', 'Name', 'Team', 'BBE', 'BIP', 'AB', 'SO', 'SF', 'SH']]

,Season,Name,Team,BBE,BIP,AB,SO,SF,SH
13848,2015,Devon Travis,TOR,176,175,217,43,1,0
15533,2013,Lyle Overbay,NYY,339,338,445,111,4,0


In [25]:
df[['Oppo', 'Cent', 'Pull', 'Soft', 'Med', 'Hard', 'GB2', 'LD2', 'FB2']] = df[['Oppo%', 'Cent%', 'Pull%', 'Soft%', 'Med%', 'Hard%', 'GB%', 'LD%', 'FB%']].mul(df['BBE'], axis=0)
# df = df[['Season', 'Name', 'Team', 'BBE', 'GB%', 'LD%', 'FB%']].merge(df[['GB', 'LD', 'FB']].div(df['BBE'], axis=0), left_index=True, right_index=True).dropna()
df[['Season', 'Name', 'Team', 'BBE', 'Oppo%', 'Cent%', 'Pull%', 'Soft%', 'Med%', 'Hard%', 'GB%', 'LD%', 'FB%']].dropna(ignore_index=True).loc[1, 'Pull%']
# df[['Season', 'Name', 'Team', 'BBE']].merge((df[['Oppo', 'Cent', 'Pull', 'Soft', 'Med', 'Hard', 'GB2', 'LD2', 'FB2']] - df[['Oppo', 'Cent', 'Pull', 'Soft', 'Med', 'Hard', 'GB2', 'LD2', 'FB2']].round()).abs(), left_index=True, right_index=True).dropna(ignore_index=True)  #[(df['GB%'].round(d) != df['GB'].round(d)) | (df['LD%'].round(d) != df['LD'].round(d)) | (df['FB%'].round(d) != df['FB'].round(d))]

0.4085

In [34]:
df.loc[9, ['Pull%', 'Cent%', 'Oppo%']] * 328

Pull%     133.988
Cent%    110.9952
Oppo%      82.984
Name: 9, dtype: object

In [73]:
df['PctSum'] = df[['GB%', 'LD%', 'FB%']].sum(axis=1)
df[df['PctSum'] > 0.0].loc[df.loc[df['PctSum'] > 0.0, 'PctSum'].sub(1.0).abs() > df.loc[df['PctSum'] > 0.0, 'PctSum'].sub(1.0).abs().quantile(0.99), ['Season', 'Name', 'Team', 'GB%', 'LD%', 'FB%', 'PctSum']]

,Season,Name,Team,GB%,LD%,FB%,PctSum
9,2023,David Peralta,LAD,0.483180,0.217125,0.299694,0.9999
14,2023,Nelson Cruz,SDP,0.438776,0.214286,0.346939,1.0001
19,2023,Roberto Pérez,SFG,0.555556,0.333333,0.111111,0.9999
27,2023,Anthony Rizzo,NYY,0.333333,0.257246,0.409420,0.9999
43,2023,Josh Donaldson,MIL,0.487805,0.097561,0.414634,0.9999
...,...,...,...,...,...,...,...
31772,2002,Salomon Torres,PIT,0.750000,0.125000,0.125000,0.9999
31773,2002,Eric Valent,PHI,0.571429,0.142857,0.285714,1.0001
31779,2002,David Lamb,MIN,0.625000,0.125000,0.250000,0.9999
31785,2002,Michael Ryan,MIN,0.777778,0.111111,0.111111,0.9999


In [85]:
df['PctSum'] = df[['Pull%', 'Cent%', 'Oppo%']].sum(axis=1)
df[df['PctSum'] > 0.0].loc[df.loc[df['PctSum'] > 0.0, 'PctSum'].sub(1.0).abs() > df.loc[df['PctSum'] > 0.0, 'PctSum'].sub(1.0).abs().quantile(0.99), ['Season', 'Name', 'Team', 'Pull%', 'Cent%', 'Oppo%', 'PctSum']]

,Season,Name,Team,Pull%,Cent%,Oppo%,PctSum
9,2023,David Peralta,LAD,0.4085,0.3384,0.2530,0.9999
27,2023,Anthony Rizzo,NYY,0.4440,0.3610,0.1949,0.9999
43,2023,Josh Donaldson,MIL,0.4634,0.3902,0.1463,0.9999
86,2023,Salvador Perez,KCR,0.4355,0.3333,0.2311,0.9999
249,2023,Jace Peterson,ARI,0.3857,0.3571,0.2571,0.9999
...,...,...,...,...,...,...,...
31382,2002,Placido Polanco,PHI,0.4308,0.3385,0.2308,1.0001
31495,2002,Eric Hinske,TOR,0.4226,0.2979,0.2794,0.9999
31532,2002,Mark Loretta,MIL,0.3420,0.2902,0.3679,1.0001
31545,2002,Chris Donnels,ARI,0.5072,0.3623,0.1304,0.9999


In [86]:
df['PctSum'] = df[['Soft%', 'Med%', 'Hard%']].sum(axis=1)
df[df['PctSum'] > 0.0].loc[df.loc[df['PctSum'] > 0.0, 'PctSum'].sub(1.0).abs() > df.loc[df['PctSum'] > 0.0, 'PctSum'].sub(1.0).abs().quantile(0.99), ['Season', 'Name', 'Team', 'Soft%', 'Med%', 'Hard%', 'PctSum']]

,Season,Name,Team,Soft%,Med%,Hard%,PctSum
785,2023,Ryan Mountcastle,BAL,0.1053,0.5573,0.3375,1.0001
2291,2022,Yonathan Daza,COL,0.1688,0.5844,0.2469,1.0001
2687,2022,Nicky Lopez,KCR,0.1849,0.5964,0.2188,1.0001
3619,2021,Enrique Hernández,BOS,0.1531,0.4914,0.3556,1.0001
4049,2021,Pat Valaika,BAL,0.1872,0.5348,0.2781,1.0001
...,...,...,...,...,...,...,...
31194,2002,Randy Velarde,OAK,0.1748,0.6408,0.1845,1.0001
31353,2002,Reggie Sanders,SFG,0.1385,0.5744,0.2872,1.0001
31424,2002,Damian Rolls,TBD,0.0811,0.7568,0.1622,1.0001
31454,2002,Ryan Ludwick,TEX,0.1053,0.6316,0.2632,1.0001


In [2]:
df = pd.read_excel('sportsref_download.xlsx')

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [7]:
pss['BBE'] = pss[['GB', 'FB', 'LD']].sum(axis=1)
pss[['Pull', 'Cent', 'Oppo', 'Soft', 'Med', 'Hard']] = pss[['Pull%', 'Center%', 'Oppo%', 'Soft%', 'Medium%', 'Hard%']].mul(pss['BBE'], axis=0)

In [21]:
pss[['Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%']].mul(pss['BBE'], axis=0)

,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%
0,237.1881,209.3082,150.5037,103.6392,294.0225,199.3383
1,260.2516,203.8848,109.8636,113.8242,311.7394,148.4364
2,259.3448,204.1468,108.5084,77.9064,300.8148,193.2788
3,167.6143,212.9050,172.4254,115.6323,315.0441,122.3789
4,202.2080,183.4072,182.3848,68.3872,350.9104,148.7024
...,...,...,...,...,...,...
104484,NaN,NaN,NaN,NaN,NaN,NaN
104485,NaN,NaN,NaN,NaN,NaN,NaN
104486,NaN,NaN,NaN,NaN,NaN,NaN
104487,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
{key: value for key, value in {"springLeague": {"id": 115, "name": "Grapefruit League", "link": "/api/v1/league/115", "abbreviation": "GL"},
  "allStarStatus": "N", "id": 144, "name": "Atlanta Braves", "link": "/api/v1/teams/144", "season": 2023,
  "venue": {"id": 4705, "name": "Truist Park", "link": "/api/v1/venues/4705"},
  "springVenue": {"id": 5380, "link": "/api/v1/venues/5380"}, "teamCode": "atl", "fileCode": "atl",
  "abbreviation": "ATL", "teamName": "Braves", "locationName": "Atlanta", "firstYearOfPlay": "1871",
  "league": {"id": 104, "name": "National League", "link": "/api/v1/league/104"},
  "division": {"id": 204, "name": "National League East", "link": "/api/v1/divisions/204"},
  "sport": {"id": 1, "link": "/api/v1/sports/1", "name": "Major League Baseball"}, "shortName": "Atlanta",
  "franchiseName": "Atlanta", "clubName": "Braves", "active": True}.items() if not isinstance(value, dict)}

{'allStarStatus': 'N',
 'id': 144,
 'name': 'Atlanta Braves',
 'link': '/api/v1/teams/144',
 'season': 2023,
 'teamCode': 'atl',
 'fileCode': 'atl',
 'abbreviation': 'ATL',
 'teamName': 'Braves',
 'locationName': 'Atlanta',
 'firstYearOfPlay': '1871',
 'shortName': 'Atlanta',
 'franchiseName': 'Atlanta',
 'clubName': 'Braves',
 'active': True}

In [40]:
abr = pd.read_csv('teamabr.csv', names=['teamID', 'leageID', 'city', 'nickname', 'startYearID', 'endYearID']).rename({'teamID': 'teamAbrRetro', 'city': 'location'}, axis=1).replace({'nickname': {'Quakers': 'Athletics'}})
abr['yearID'] = abr.apply(lambda x: range(x['startYearID'], x['endYearID'] + 1), axis=1)
abr = abr.drop(['startYearID', 'endYearID', 'leageID'], axis=1).explode('yearID')
abr = abr[(abr['teamAbrRetro'] != 'CN1') | (abr['yearID'] != 1880)]  # Cincinnati Reds did not play in 1880
# abr.loc[(abr['yearID'] == 1891) & (abr['teamAbrRetro'] == 'PH4'), 'teamAbrRetro'] = 'PHP'
abr = pd.concat([abr, abr[abr['yearID'] == 2021].assign(yearID=2022), abr[abr['yearID'] == 2021].assign(yearID=2023)])  # duplicate 2021 data for 2022 and 2023
abr

,teamAbrRetro,location,nickname,yearID
46,PH4,Philadelphia,Athletics,1882
46,PH4,Philadelphia,Athletics,1883
46,PH4,Philadelphia,Athletics,1884
46,PH4,Philadelphia,Athletics,1885
46,PH4,Philadelphia,Athletics,1886
46,PH4,Philadelphia,Athletics,1887
46,PH4,Philadelphia,Athletics,1888
46,PH4,Philadelphia,Athletics,1889
46,PH4,Philadelphia,Athletics,1890
46,PH4,Philadelphia,Athletics,1891


In [42]:
home_parks = pd.read_csv('baseballdatabank-2022.2/core/HomeGames.csv')[['year.key', 'team.key', 'park.key', 'games']].rename({'year.key': 'yearID', 'team.key': 'teamAbrLahman', 'park.key': 'parkID'}, axis=1)
home_parks = home_parks[home_parks.groupby(['yearID', 'teamAbrLahman'])['games'].transform('max') == home_parks['games']].drop('games', axis=1)
home_parks = pd.concat([home_parks, home_parks[home_parks['yearID'] == 2021].assign(yearID=2022), home_parks[home_parks['yearID'] == 2021].assign(yearID=2023)])
home_parks

,yearID,teamAbrLahman,parkID
0,1871,BS1,BOS01
2,1871,CH1,CHI01
5,1871,CL1,CLE01
6,1871,FW1,FOR01
7,1871,NY2,NYC01
...,...,...,...
3187,2023,SLN,STL10
3188,2023,TBA,STP01
3189,2023,TEX,ARL03
3193,2023,TOR,TOR02


In [64]:
lahman = pd.read_csv('baseballdatabank-2022.2/core/Teams.csv')[['yearID', 'lgID', 'teamID', 'teamIDlahman45', 'teamIDretro', 'franchID', 'divID', 'name']].fillna({'lgID': 'NA'}).rename({'teamID': 'teamAbrLahman', 'teamIDlahman45': 'teamAbrLahman45', 'franchID': 'franchIDLahman', 'teamIDretro': 'teamAbrRetro', 'name': 'teamName'}, axis=1).replace(ABR_CORRECTIONS)

lahman = pd.concat([lahman, lahman[lahman['yearID'] == 2021].assign(yearID=2022), lahman[lahman['yearID'] == 2021].assign(yearID=2023)])  # duplicate 2021 data for 2022 and 2023
lahman.loc[lahman['franchIDLahman'] == 'PHP', ['teamAbrLahman', 'teamAbrLahman45', 'teamAbrRetro']] = 'PHP'

abr = pd.read_csv('teamabr.csv', names=['teamID', 'leageID', 'city', 'nickname', 'startYearID', 'endYearID']).rename({'teamID': 'teamAbrRetro', 'city': 'location'}, axis=1)
abr['yearID'] = abr.apply(lambda x: range(x['startYearID'], x['endYearID'] + 1), axis=1)
abr = abr.drop(['startYearID', 'endYearID', 'leageID'], axis=1).explode('yearID')
abr = abr[(abr['teamAbrRetro'] != 'CN1') | (abr['yearID'] != 1880)]  # Cincinnati Reds did not play in 1880
abr.loc[(abr['yearID'] == 1891) & (abr['teamAbrRetro'] == 'PH4'), 'teamAbrRetro'] = 'PHP'
abr = pd.concat([abr, abr[abr['yearID'] == 2021].assign(yearID=2022), abr[abr['yearID'] == 2021].assign(yearID=2023)])  # duplicate 2021 data for 2022 and 2023

home = pd.read_csv('baseballdatabank-2022.2/core/HomeGames.csv')[['year.key', 'team.key', 'park.key', 'games']].rename({'year.key': 'yearID', 'team.key': 'teamAbrLahman', 'park.key': 'parkID'}, axis=1)
home = home[home.groupby(['yearID', 'teamAbrLahman'])['games'].transform('max') == home['games']].drop('games', axis=1)
home = pd.concat([home, home[home['yearID'] == 2021].assign(yearID=2022), home[home['yearID'] == 2021].assign(yearID=2023)])

df = df.merge(lahman, on=['yearID', 'franchIDLahman']).merge(abr, on=['yearID', 'teamAbrRetro']).merge(home, how='left', on=['yearID', 'teamAbrLahman']).rename({'parkID': 'mainHomeParkID'}, axis=1)

df.loc[(df['yearID'] == 1880) & (df['franchIDFG'] == 1028), ['franchIDFG', 'franchIDLahman', 'teamAbrLahman', 'teamAbrLahman45']] = [1091, 'CNS', 'CN4', 'CN4']

teams = df[['yearID', 'franchIDFG', 'teamAbrFG', 'teamAbrLahman', 'teamAbrLahman45', 'teamName', 'location', 'nickname', 'lgID', 'divID', 'mainHomeParkID']]
franchises = df[['franchIDFG', 'franchIDLahman', 'yearID', 'teamName']]
franchises.groupby('franchIDLahman')['franchIDFG'].nunique().sort_values(ascending=False)
# franch[franch['franchIDFG'].isin([16, 17, 1002, 1033, 1053, 1062, 1070])].sort_values(['franchIDFG', 'yearID'], ignore_index=True)
df[df['franchIDFG'].isin([1, 16, 2, 10, 24, 22, 20, 1051, 1006, 13, 12, 9, 8, 1082, 30, 1026, 1062, 23])]

,yearID,teamAbrFG,franchIDFG,franchIDLahman,lgID,teamAbrLahman,teamAbrLahman45,teamAbrRetro,divID,teamName,location,nickname,mainHomeParkID
0,1871,BSN,16,ATL,NA,BS1,BS1,BS1,NaN,Boston Red Stockings,Boston,Braves,BOS01
9,1872,BSN,16,ATL,NA,BS1,BS1,BS1,NaN,Boston Red Stockings,Boston,Braves,BOS01
20,1873,BSN,16,ATL,NA,BS1,BS1,BS1,NaN,Boston Red Stockings,Boston,Braves,BOS01
29,1874,BSN,16,ATL,NA,BS1,BS1,BS1,NaN,Boston Red Stockings,Boston,Braves,BOS01
37,1875,BSN,16,ATL,NA,BS1,BS1,BS1,NaN,Boston Red Stockings,Boston,Braves,BOS01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3032,2023,SFG,30,SFG,NL,SFN,SFN,SFN,W,San Francisco Giants,San Francisco,Giants,SFO03
3034,2023,NYY,9,NYY,AL,NYA,NYA,NYA,E,New York Yankees,New York,Yankees,NYC21
3040,2023,MIA,20,FLA,NL,MIA,FLO,MIA,E,Miami Marlins,Miami,Marlins,MIA02
3041,2023,WSN,24,WSN,NL,WAS,MON,WAS,E,Washington Nationals,Washington,Nationals,WAS11


In [65]:
t = team_ids()
t.loc[t['teamIDfg'] == 16, ['teamIDfg', 'teamIDBR']].drop_duplicates()

,teamIDfg,teamIDBR
0,16,BSN
1192,16,MLN
1417,16,ATL


In [34]:
df[df['franchIDFG'].isin([1060, 1062])].sort_values('franchIDFG')

,yearID,teamAbrFG,franchIDFG,franchIDLahman,lgID,teamAbrLahman,teamAbrLahman45,teamAbrRetro,divID,teamName
102,1882,PHA,1060,PHA,AA,PH4,PH4,PH4,NaN,Philadelphia Athletics
115,1883,PHA,1060,PHA,AA,PH4,PH4,PH4,NaN,Philadelphia Athletics
132,1884,PHA,1060,PHA,AA,PH4,PH4,PH4,NaN,Philadelphia Athletics
160,1885,PHA,1060,PHA,AA,PH4,PH4,PH4,NaN,Philadelphia Athletics
183,1886,PHA,1060,PHA,AA,PH4,PH4,PH4,NaN,Philadelphia Athletics
197,1887,PHA,1060,PHA,AA,PH4,PH4,PH4,NaN,Philadelphia Athletics
206,1888,PHA,1060,PHA,AA,PH4,PH4,PH4,NaN,Philadelphia Athletics
223,1889,PHA,1060,PHA,AA,PH4,PH4,PH4,NaN,Philadelphia Athletics
258,1890,PHA,1060,PHA,AA,PH4,PH4,PH4,NaN,Philadelphia Athletics
253,1890,PHP,1062,PHP,PL,PHP,PHP,PHP,NaN,Philadelphia Athletics


In [37]:
franchises = pd.read_csv('baseballdatabank-2022.2/core/Teams.csv')
franchises.groupby('teamID')['franchID'].nunique().sort_values(ascending=False)

teamID
WAS    2
PH4    2
ALT    1
PHI    1
OAK    1
      ..
COL    1
DET    1
DTN    1
ELI    1
WSU    1
Name: franchID, Length: 149, dtype: int64

In [40]:
franchises.loc[franchises['teamID'].isin(['WAS', 'PH4']), ['teamID', 'franchID']].drop_duplicates().sort_values('teamID')

,teamID,franchID
102,PH4,PHA
274,PH4,PHQ
290,WAS,WAS
2504,WAS,WSN


In [45]:
franchises[franchises['yearID'] == 1891]

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
262,1891,AA,BL3,BLO,NaN,4,139,NaN,71,64,...,103,0.915,Baltimore Orioles,Union Park,NaN,101,100,BAL,BL3,BL3
263,1891,NL,BRO,LAD,NaN,6,137,NaN,61,76,...,73,0.924,Brooklyn Grooms,Eastern Park,181477.0,99,99,BRO,BRO,BRO
264,1891,AA,BS2,BRS,NaN,1,139,NaN,93,42,...,115,0.934,Boston Reds,Congress Street Grounds,NaN,103,99,BOS,BS2,BS2
265,1891,NL,BSN,ATL,NaN,1,140,NaN,87,51,...,96,0.938,Boston Beaneaters,South End Grounds II,184472.0,112,109,BSN,BSN,BSN
266,1891,NL,CHN,CHC,NaN,2,137,NaN,82,53,...,119,0.932,Chicago Colts,South Side Park I,181431.0,106,105,CHC,CHN,CHN
267,1891,NL,CIN,CIN,NaN,7,138,NaN,56,81,...,101,0.931,Cincinnati Reds,League Park I in Cincinnati,97500.0,101,101,CIN,CIN,CIN
268,1891,NL,CL4,CLV,NaN,5,141,NaN,65,74,...,86,0.920,Cleveland Spiders,League Park I,132000.0,104,103,CLV,CL4,CL4
269,1891,AA,CL6,CLS,NaN,6,138,NaN,61,76,...,126,0.935,Columbus Solons,Recreation Park II,NaN,95,93,COL,CL6,CL6
270,1891,AA,CN3,CKK,NaN,7,102,NaN,43,57,...,68,0.913,Cincinnati Kelly's Killers,East End Park,NaN,108,110,CKK,CN3,CN3
271,1891,AA,LS2,LOU,NaN,8,138,NaN,54,83,...,112,0.922,Louisville Colonels,Eclipse Park I,NaN,97,97,LOU,LS2,LS2


In [27]:
df.groupby('franchID')['franchAbrLahman'].nunique().sort_values(ascending=False)

franchID
1033    2
1002    2
1070    2
1053    2
17      2
       ..
1007    1
1006    1
1005    1
1004    1
1091    1
Name: franchAbrLahman, Length: 115, dtype: int64

In [46]:
team_ids()

,yearID,lgID,teamID,franchID,teamIDfg,teamIDBR,teamIDretro
0,1876,NL,BSN,ATL,16,BSN,BSN
1,1876,NL,CHN,CHC,17,CHC,CHN
2,1876,NL,CN1,CNR,1028,CIN,CN1
3,1876,NL,HAR,HAR,1033,HAR,HAR
4,1876,NL,LS1,LGR,1043,LOU,LS1
...,...,...,...,...,...,...,...
2930,2021,NL,PIT,PIT,27,PIT,PIT
2931,2021,NL,SDN,SDP,29,SDP,SDN
2932,2021,NL,SFN,SFG,30,SFG,SFN
2933,2021,NL,SLN,STL,28,STL,SLN


In [33]:
df[df['teamAbrLahman'] == 'CN3']

,yearID,teamAbrFG,franchID,franchAbrLahman,lgID,teamAbrLahman,teamAbrLahman45,teamAbrRetro,divID,teamName,location,nickname,mainHomeParkID
277,1891,CKE,1023,CKK,AA,CN3,CN3,CN3,NaN,Cincinnati Kelly's Killers,Cincinnati,Kelly's Killers,PEN01


In [3]:
new_parks = pd.read_csv('Ballpark Database 2019 Final Files/Home Main Data With Parks Breakout.csv')[['Year', 'TeamID', 'Park_ID']].rename({'Year': 'yearID', 'TeamID': 'teamID', 'Park_ID': 'parkID'}, axis=1).merge(pd.read_csv('Ballpark Database 2019 Final Files/Parks.csv')[['PARKID', 'NAME']].rename({'PARKID': 'parkID', 'NAME': 'parkName'}, axis=1), on='parkID')
new_parks.groupby(['yearID', 'teamID'])['parkID'].nunique().sort_values(ascending=False)
# new_parks[['yearID', 'teamID']].drop_duplicates().shape[0] + 4 * 30
# park_mapping = pd.read_csv('Ballpark Database 2019 Final Files/Retrosheet_BBDB_Team_XRef.csv')
# new_parks[~new_parks['TeamID'].isin(df['teamAbrLahman'])] # = new_parks.merge(park_mapping, left_on='TeamID', right_on='BBDBID')

yearID  teamID
1898    CL4       7
1902    CLE       5
1871    WS3       4
1885    BFN       3
1888    PH4       3
                 ..
1940    SLN       1
        WS1       1
1941    BOS       1
        BRO       1
2019    WAS       1
Name: parkID, Length: 2925, dtype: int64

In [25]:
war = pd.read_csv('batterwar.csv').merge(pd.read_csv('pitcherwar.csv'), how='outer', on='PlayerId', suffixes=('_batting', '_pitching')).fillna(0.0).assign(tWAR=lambda x: x['WAR_batting'] + x['aWAR']).merge(pd.read_csv('fgplayerids.csv').astype({'MLBAMID': 'Int64'}), on='PlayerId')
war[['Name', 'WAR_batting', 'aWAR', 'tWAR']].rename({'WAR_batting': 'ppWAR', 'aWAR': 'pWAR', 'tWAR': 'WAR'}, axis=1).sort_values('WAR', ascending=False, ignore_index=True)

,Name,ppWAR,pWAR,WAR
0,Babe Ruth,167.017911,18.864698,185.882609
1,Barry Bonds,164.448151,0.000000,164.448151
2,Cy Young,0.759004,155.322210,156.081214
3,Willie Mays,149.853499,0.000000,149.853499
4,Ty Cobb,149.076666,0.000000,149.076666
...,...,...,...,...
7271,Vic Harris,-7.142873,0.000000,-7.142873
7272,Jim Levey,-8.000576,0.000000,-8.000576
7273,Doug Flynn,-8.367821,0.000000,-8.367821
7274,Ryan Doumit,-8.621528,0.000000,-8.621528


In [5]:
home[home['yearID'] == 2022]

,yearID,teamAbrLahman,parkID,startDate,endDate,games


In [50]:
df[df['yearID'] == 1884].sort_values('teamAbrLahman')

,yearID,teamAbrFG,franchID,franchAbrLahman,lgID,teamAbrLahman,teamAbrLahman45,teamAbrRetro,divID,teamName,park,location,nickname
151,1884,ALT,1001,ALT,UA,ALT,ALT,ALT,NaN,Altoona Mountain City,NaN,Altoona,Mountain Citys
135,1884,BUF,1015,BUF,NL,BFN,BFN,BFN,NaN,Buffalo Bisons,Olympic Park I,Buffalo,Bisons
137,1884,BAL,1006,BLO,AA,BL2,BL2,BL2,NaN,Baltimore Orioles,Oriole Park,Baltimore,Orioles
139,1884,BAL,1008,BLU,UA,BLU,BLU,BLU,NaN,Baltimore Monumentals,NaN,Baltimore,Monumentals
144,1884,BRO,22,LAD,AA,BR3,BR3,BR3,NaN,Brooklyn Atlantics,Washington Park I,Brooklyn,Dodgers
131,1884,BSN,16,ATL,NL,BSN,BSN,BSN,NaN,Boston Beaneaters,South End Grounds I,Boston,Braves
138,1884,BSU,1011,BRD,UA,BSU,BSU,BSU,NaN,Boston Reds,NaN,Boston,Reds
125,1884,CHI,17,CHC,NL,CHN,CHN,CHN,NaN,Chicago White Stockings,Lake Front Park II,Chicago,Cubs
140,1884,CUN,1030,CPI,UA,CHU,CHU,CHU,NaN,Chicago/Pittsburgh (Union League),NaN,Chicago-Pittsburgh,Browns
141,1884,CLE,1018,CBL,NL,CL2,CL2,CL2,NaN,Cleveland Blues,Kennard Street Park,Cleveland,Spiders


In [29]:
url = 'https://www.fangraphs.com/leaders/major-league?pos=all&stats={stats}&lg=aa&lg=al&lg=fl&lg=na&lg=nl&lg=pl&lg=ua&qual=0&type=8&month=0&ind=0&team=0%2Cts&rost=&age=&filter=&players=0&startdate=&enddate=&season1=1898&season=1898'

df = pd.DataFrame(json.loads(BeautifulSoup(requests.get(url.format(stats='bat')).content, 'html.parser', from_encoding='utf_8').find('script', type='application/json').text)['props']['pageProps']['dehydratedState']['queries'][0]['state']['data']['data']).replace('- - -', np.nan).drop(['Name', 'PlayerName', 'PlayerNameRoute', 'Team', 'TeamNameAbb', 'SeasonMin', 'SeasonMax', 'Bats', 'Throws', 'xMLBAMID', 'playerid', 'Age', 'AgeR', 'G', 'GS', 'PA', 'IP', 'Pos', 'Inn'], axis=1, errors='ignore').rename({'TG': 'G', 'TPA': 'PA', 'TIP': 'IP', 'TInn': 'Inn'}, axis=1)


for col in {'IP', 'Inn'} & set(df.columns):
    df[col] = df[col].mul(3).round().div(3)
    
df.head()

,Season,AB,H,1B,2B,3B,HR,R,RBI,BB,...,Barrels,Barrel%,maxEV,HardHit,HardHit%,Q,G,PA,TeamName,teamid
0,1898,5242,1584,1341,154,77,12,933,757,519,...,None,None,None,None,None,39.2157,153,6000,BAL,1006
1,1898,5276,1531,1233,190,55,53,872,761,405,...,None,None,None,None,None,38.4671,152,5847,BSN,16
2,1898,5219,1431,1154,175,84,18,828,656,476,...,None,None,None,None,None,38.6776,152,5879,CHI,17
3,1898,5118,1431,1079,238,81,33,823,706,472,...,None,None,None,None,None,38.5267,150,5779,PHI,26
4,1898,5334,1448,1121,207,101,19,831,688,455,...,None,None,None,None,None,38.1146,157,5984,CIN,18


In [12]:
df['Inn'].mul(3).round().div(3)

0     430.333333
1     419.000000
2     515.666667
3     336.000000
4     199.000000
5     259.333333
6     211.000000
7      99.000000
8     512.000000
9      79.000000
10    225.000000
Name: Inn, dtype: float64

In [6]:
franchises = pd.read_csv('baseballdatabank-2022.2/core/TeamsFranchises.csv')
lahman = pd.read_csv('baseballdatabank-2022.2/core/Teams.csv')
parks = pd.read_csv('baseballdatabank-2022.2/core/Parks.csv').rename({'park.key': 'parkID', 'park.name': 'name', 'park.alias': 'alias'}, axis=1)

In [10]:
franchises

,franchID,franchName,active,NAassoc
0,ALT,Altoona Mountain City,N,NaN
1,ANA,Los Angeles Angels of Anaheim,Y,NaN
2,ARI,Arizona Diamondbacks,Y,NaN
3,ATH,Philadelphia Athletics,N,PNA
4,ATL,Atlanta Braves,Y,BNA
...,...,...,...,...
115,WNL,Washington Nationals,N,NaN
116,WNT,Washington Nationals,NaN,NaN
117,WOR,Worcester Ruby Legs,N,NaN
118,WSN,Washington Nationals,Y,NaN


In [58]:
teams.sort_values('yearID', ascending=False)

,yearID,lgID,teamID,franchID,teamIDfg,teamIDBR,teamIDretro
2934,2021,NL,WAS,WSN,24,WSN,WAS
2919,2021,AL,TOR,TOR,14,TOR,TOR
2905,2021,AL,BAL,BAL,2,BAL,BAL
2906,2021,AL,BOS,BOS,3,BOS,BOS
2907,2021,AL,CHA,CHW,4,CHW,CHA
...,...,...,...,...,...,...,...
1,1876,NL,CHN,CHC,17,CHC,CHN
5,1876,NL,NY3,NYU,1053,NYU,NY3
6,1876,NL,PHN,ATH,1002,ATH,PHN
7,1876,NL,SL3,SBS,1070,STL,SL3


In [51]:
lahman[lahman['yearID'] >= 1876].shape

(2935, 48)

In [7]:
teams.merge(lahman[['yearID', 'teamID', 'divID', 'name', 'park']], on=['yearID', 'teamID']).merge()

,yearID,lgID,teamID,franchID,teamIDfg,teamIDBR,teamIDretro,divID,name,park
0,1876,NL,BSN,ATL,16,BSN,BSN,NaN,Boston Red Caps,South End Grounds I
1,1876,NL,CHN,CHC,17,CHC,CHN,NaN,Chicago White Stockings,23rd Street Grounds
2,1876,NL,CN1,CNR,1028,CIN,CN1,NaN,Cincinnati Reds,Avenue Grounds
3,1876,NL,HAR,HAR,1033,HAR,HAR,NaN,Hartford Dark Blues,Hartford Ball Club Grounds
4,1876,NL,LS1,LGR,1043,LOU,LS1,NaN,Louisville Grays,Louisville Baseball Park
...,...,...,...,...,...,...,...,...,...,...
2930,2021,NL,PIT,PIT,27,PIT,PIT,C,Pittsburgh Pirates,PNC Park
2931,2021,NL,SDN,SDP,29,SDP,SDN,W,San Diego Padres,Petco Park
2932,2021,NL,SFN,SFG,30,SFG,SFN,W,San Francisco Giants,Oracle Park
2933,2021,NL,SLN,STL,28,STL,SLN,C,St. Louis Cardinals,Busch Stadium III


In [19]:
lahman[lahman['park'].isna()]

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
124,1884,UA,ALT,ALT,NaN,10,25,NaN,6,19,...,4,0.862,Altoona Mountain City,NaN,NaN,101,109,ALT,ALT,ALT
127,1884,UA,BLU,BLU,NaN,4,105,NaN,58,47,...,53,0.872,Baltimore Monumentals,NaN,NaN,109,109,BLU,BLU,BLU
130,1884,UA,BSU,BRD,NaN,5,109,NaN,58,51,...,39,0.868,Boston Reds,NaN,NaN,99,98,BOS,BSU,BSU
132,1884,UA,CHU,CPI,NaN,6,92,NaN,41,50,...,38,0.882,Chicago/Pittsburgh (Union League),NaN,NaN,98,99,CPI,CHU,CHU
136,1884,UA,CNU,COR,NaN,3,103,NaN,69,36,...,45,0.882,Cincinnati Outlaw Reds,NaN,NaN,111,107,COR,CNU,CNU
139,1884,UA,KCU,KCU,NaN,11,82,NaN,16,63,...,51,0.861,Kansas City Cowboys,NaN,NaN,87,92,KCC,KCU,KCU
141,1884,UA,MLU,MLU,NaN,2,12,NaN,8,4,...,4,0.892,Milwaukee Brewers,NaN,NaN,60,60,MIL,MLU,MLU
146,1884,UA,PHU,PHK,NaN,8,67,NaN,21,46,...,36,0.841,Philadelphia Keystones,NaN,NaN,91,94,PHK,PHU,PHU
151,1884,UA,SLU,SLM,NaN,1,113,NaN,94,19,...,79,0.888,St. Louis Maroons,NaN,NaN,99,98,SLM,SLU,SLU
152,1884,UA,SPU,STP,NaN,9,9,NaN,2,6,...,6,0.872,St. Paul White Caps,NaN,NaN,60,60,STP,SPU,SPU


In [52]:
abr = pd.read_csv('teamabr.csv', names=['teamID', 'leageID', 'city', 'nickname', 'startYearID', 'endYearID']).query('startYearID > 1876')
abr['yearID'] = abr.apply(lambda x: [year for year in range(x['startYearID'], x['endYearID'] + 1) if year >= 1876], axis=1)
abr = abr.drop(['startYearID', 'endYearID', 'leageID'], axis=1).explode('yearID').rename({'teamID': 'teamIDRetro', 'city': 'location'}, axis=1).sort_values(['teamIDRetro', 'yearID'])
abr

,teamIDRetro,location,nickname,yearID
57,ALT,Altoona,Mountain Citys,1884
122,ANA,Anaheim,Angels,1997
122,ANA,Anaheim,Angels,1998
122,ANA,Anaheim,Angels,1999
122,ANA,Anaheim,Angels,2000
...,...,...,...,...
97,WSN,Washington,Senators,1896
97,WSN,Washington,Senators,1897
97,WSN,Washington,Senators,1898
97,WSN,Washington,Senators,1899


In [50]:
nicknames = pd.read_csv('CurrentNames.csv', names=['retroID', 'altID', 'leagueID', 'divisionID', 'location', 'nickname', 'altNickname', 'firstGameDate', 'lastGameDate', 'city', 'state'], parse_dates=['firstGameDate', 'lastGameDate'])

nicknames

,retroID,altID,leagueID,divisionID,location,nickname,altNickname,firstGameDate,lastGameDate,city,state
0,ANA,LAA,AL,NaN,Los Angeles,Angels,NaN,1961-04-11,1965-09-01,Los Angeles,CA
1,ANA,CAL,AL,NaN,California,Angels,NaN,1965-09-02,1968-09-29,Anaheim,CA
2,ANA,CAL,AL,W,California,Angels,NaN,1969-04-08,1996-09-29,Anaheim,CA
3,ANA,ANA,AL,W,Anaheim,Angels,NaN,1997-04-02,2004-10-03,Anaheim,CA
4,ANA,ANA,AL,W,Los Angeles,Angels,NaN,2005-04-05,NaT,Anaheim,CA
...,...,...,...,...,...,...,...,...,...,...,...
119,TEX,WS2,AL,E,Washington,Senators,NaN,1969-04-07,1971-09-30,Washington,DC
120,TEX,TEX,AL,W,Texas,Rangers,NaN,1972-04-15,NaT,Arlington,TX
121,TOR,TOR,AL,E,Toronto,Blue Jays,NaN,1977-04-07,NaT,Toronto,ON
122,WAS,MON,NL,E,Montreal,Expos,NaN,1969-04-08,2004-10-03,Montreal,QC


In [2]:
from pybaseball import team_game_logs

team_game_logs(2019, "ATL")

/usr/local/Caskroom/miniforge/base/envs/default/lib/python3.12/site-packages/pybaseball/team_game_logs.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


,Game,Date,Home,Opp,Rslt,PA,AB,R,H,2B,...,SB,CS,BA,OBP,SLG,OPS,LOB,NumPlayers,Thr,OppStart
0,1,Mar 28,False,PHI,"L,4-10",36,30,4,7,1,...,1,1,0.233,0.361,0.367,0.728,5,15,R,A.Nola(67)
1,2,Mar 30,False,PHI,"L,6-8",40,36,6,10,2,...,1,0,0.258,0.355,0.470,0.825,7,17,R,N.Pivetta(35)
2,3,Mar 31,False,PHI,"L,1-5",36,29,1,4,0,...,1,2,0.221,0.339,0.368,0.708,8,14,R,J.Arrieta(62)
3,4,Apr 1,True,CHC,"W,8-0",43,36,8,12,3,...,0,0,0.252,0.368,0.427,0.795,11,15,R,K.Hendricks(26)
4,5,Apr 3,True,CHC,"W,6-4",39,30,6,8,1,...,1,0,0.255,0.376,0.441,0.817,9,16,L,J.Lester(56)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,158,Sep 24,False,KCR,"L,6-9",41,37,6,11,3,...,0,0,0.258,0.336,0.454,0.790,8,13,L,D.Duffy(48)
158,159,Sep 25,False,KCR,"W,10-2",48,40,10,14,4,...,0,0,0.258,0.337,0.454,0.791,11,13,L,M.Montgomery(47)
159,160,Sep 27,False,NYM,"L,2-4",39,35,2,9,0,...,2,0,0.258,0.337,0.453,0.790,10,15,R,M.Stroman(59)
160,161,Sep 28,False,NYM,"L,0-3",37,31,0,4,0,...,0,0,0.258,0.337,0.451,0.788,10,14,L,S.Matz(70)


In [5]:
from pybaseball import statcast_single_game

statcast_single_game(529429).col

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
159,CH,2018-03-30,88.1,-2.62,6.21,"Pederson, Joc",592626,519326,field_out,hit_into_play,...,1,1,0,0,1,Standard,Standard,232.0,-0.044,-0.100
160,SL,2018-03-30,84.2,-3.01,5.80,"Forsythe, Logan",523253,519326,strikeout,called_strike,...,1,1,0,0,1,Standard,Standard,91.0,-0.064,-0.212
164,SL,2018-03-30,85.3,-3.08,5.74,"Forsythe, Logan",523253,519326,NaN,foul,...,1,1,0,0,1,Standard,Standard,94.0,0.000,0.000
166,FF,2018-03-30,94.8,-2.75,5.66,"Forsythe, Logan",523253,519326,NaN,ball,...,1,1,0,0,1,Standard,Standard,194.0,0.000,0.064
175,FF,2018-03-30,95.3,-2.96,5.62,"Forsythe, Logan",523253,519326,NaN,foul,...,1,1,0,0,1,Standard,Standard,189.0,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,KC,2018-03-30,84.2,1.93,5.53,"Panik, Joe",605412,622072,NaN,ball,...,0,0,0,0,0,Standard,Standard,288.0,0.000,0.037
206,SI,2018-03-30,91.5,1.84,5.72,"Panik, Joe",605412,622072,NaN,called_strike,...,0,0,0,0,0,Standard,Standard,135.0,0.000,-0.034
213,SI,2018-03-30,91.5,1.67,5.82,"Panik, Joe",605412,622072,NaN,ball,...,0,0,0,0,0,Standard,Standard,133.0,0.000,0.027
217,CH,2018-03-30,85.0,2.00,5.59,"Jackson, Austin",457706,622072,field_out,hit_into_play,...,0,0,0,0,0,Standard,Standard,108.0,0.022,-0.196


In [7]:
df['yearID'].value_counts().sort_index()

yearID
1871     9
1872    11
1873     9
1874     8
1875    13
        ..
2019    30
2020    30
2021    30
2022    30
2023    30
Name: count, Length: 153, dtype: int64